In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from IPython.display import display, Markdown
import matplotlib.colors as mcolors

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-team'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Calculate per-game statistics before dropping 'G'
    if 'G' in team_stats.columns:
        for col in per_game_columns:
            if col in team_stats.columns:  # Ensure the column exists
                team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Drop the 'G' and 'MP' columns after calculations
    team_stats.drop(columns=['G', 'MP'], inplace=True, errors='ignore')

    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTM/g', 'FTA/g', 'FT%', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Filter rows for Raptors 905 and League Average
    filtered_teams = team_stats.loc[team_stats['Team'].isin(['Raptors 905', 'League Average'])]
    
else:
    print("Table with id 'team-stats-sc-team' not found.")

# Quit the driver
driver.quit()

filtered_teams

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTM/g,FTA/g,FT%,AST/g,STL/g,BLK/g,PF/g,PTS/g
10,Raptors 905,43.500,91.625,0.475,12.750,36.875,0.346,30.750,54.750,0.562,8.375,13.0,0.644,27.375,12.25,7.25,16.125,113.250
31,League Average,42.375,93.625,0.453,13.375,39.500,0.338,29.125,54.125,0.537,11.250,15.0,0.749,24.625,9.75,6.00,10.875,116.625


In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-team'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Create per-game columns
    for col in per_game_columns:
        if col in team_stats.columns:  # Ensure the column exists
            team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg', 'G', 'MP']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Exclude the "League Average" row
    team_stats = team_stats[team_stats['Team'] != 'League Average'].reset_index(drop=True)
    
    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTM/g', 'FTA/g', 'FT%', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Calculate ranks and append ranks to numeric columns for display
    invert_cols = ['PF/g']  # Specify columns where ranking should be ascending
    df_with_ranks = team_stats.copy()
    for col in numeric_cols:
        # Calculate ranks (invert order for specified columns)
        if col in invert_cols:
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
        else:
            ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        
        # Append ranks to column values for display
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
    
    # Color coding logic
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            if col in invert_cols:
                # Invert normalization for specified columns
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            else:
                # Normal normalization
                norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply the coloring logic to the DataFrame with ranks
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

else:
    print("Table with id 'team-stats-sc-team' not found.")

# Quit the driver
driver.quit()

styled_df

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTM/g,FTA/g,FT%,AST/g,STL/g,BLK/g,PF/g,PTS/g
0,Valley Suns,44.625 (2),91.375 (9),0.488 (2),18.75 (1),44.0 (4),0.426 (1),25.875 (23),47.375 (25),0.546 (13),10.375 (16),14.0 (16),0.741 (20),25.75 (6),8.25 (23),4.125 (26),17.375 (31),125.125 (1)
1,Westchester Knicks,46.5 (1),98.25 (1),0.473 (7),11.0 (24),34.25 (25),0.321 (21),35.5 (1),64.0 (1),0.555 (8),9.5 (22),14.375 (13),0.661 (29),25.75 (6),10.0 (11),8.0 (3),12.25 (23),120.125 (2)
2,San Diego Clippers,41.5 (8),91.5 (7),0.454 (13),12.625 (15),36.125 (18),0.349 (11),28.875 (10),55.375 (6),0.521 (24),14.125 (2),20.375 (1),0.693 (27),22.875 (17),12.625 (3),3.75 (29),4.25 (2),118.625 (3)
3,Rip City Remix,42.75 (6),94.75 (2),0.451 (15),14.25 (7),45.0 (2),0.317 (25),28.5 (11),49.75 (21),0.573 (3),10.625 (14),13.625 (18),0.78 (8),20.875 (26),8.5 (19),6.25 (11),14.5 (28),117.0 (4)
4,Sioux Falls Skyforce,44.0 (3),87.444 (20),0.503 (1),12.0 (19),34.222 (26),0.351 (8),32.0 (3),53.222 (12),0.601 (1),10.0 (19),12.889 (23),0.776 (9),29.333 (1),10.333 (7),5.222 (19),9.889 (14),116.778 (5)
5,Motor City Cruise,43.111 (5),89.889 (12),0.48 (4),13.222 (10),35.556 (20),0.372 (3),29.889 (8),54.333 (10),0.55 (11),9.444 (24),12.0 (26),0.787 (6),25.667 (8),10.111 (9),4.889 (21),5.222 (3),115.556 (6)
6,Stockton Kings,41.375 (9),87.5 (19),0.473 (7),15.625 (4),38.125 (11),0.41 (2),25.75 (24),49.375 (23),0.522 (22),10.125 (18),13.5 (20),0.75 (17),26.25 (4),7.125 (29),6.0 (13),12.875 (24),115.125 (7)
7,Salt Lake City Stars,41.375 (9),91.5 (7),0.452 (14),11.25 (21),35.0 (21),0.321 (21),30.125 (7),56.5 (5),0.533 (17),12.25 (5),15.25 (8),0.803 (5),23.0 (15),10.375 (6),3.75 (29),9.625 (13),114.75 (8)
8,Iowa Wolves,42.2 (7),87.6 (18),0.482 (3),9.6 (29),28.5 (30),0.337 (14),32.6 (2),59.1 (3),0.552 (9),11.9 (8),16.1 (5),0.739 (22),25.1 (9),10.1 (10),5.9 (14),13.6 (25),114.4 (9)
9,Long Island Nets,41.333 (11),92.667 (5),0.446 (18),10.889 (25),34.778 (22),0.313 (26),30.444 (5),57.889 (4),0.526 (21),11.889 (9),15.222 (9),0.781 (7),24.556 (10),8.444 (20),5.333 (18),14.667 (29),113.778 (10)


In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-team'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Create per-game columns
    for col in per_game_columns:
        if col in team_stats.columns:  # Ensure the column exists
            team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg', 'G', 'MP']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Exclude the "League Average" row
    team_stats = team_stats[team_stats['Team'] != 'League Average'].reset_index(drop=True)
    
    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTM/g', 'FTA/g', 'FT%', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Filter the row for Raptors 905 only
    raptors_row = team_stats.loc[team_stats['Team'] == 'Raptors 905']
    
    # Calculate ranks and append ranks to numeric columns for display
    invert_cols = ['PF/g']  # Specify columns where ranking should be ascending
    df_with_ranks = team_stats.copy()
    for col in numeric_cols:
        # Calculate ranks (invert order for specified columns)
        if col in invert_cols:
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
        else:
            ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        
        # Append ranks to column values for display
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
    
    # Apply color coding logic to Raptors 905 row
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            if col in invert_cols:
                # Invert normalization for specified columns
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            else:
                # Normal normalization
                norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Style the Raptors 905 row
    styled_df = df_with_ranks.loc[df_with_ranks['Team'] == 'Raptors 905'].style.apply(
        color_cells, subset=numeric_cols, axis=1
    ).format(precision=3)

else:
    print("Table with id 'team-stats-sc-team' not found.")

# Quit the driver
driver.quit()

styled_df

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTM/g,FTA/g,FT%,AST/g,STL/g,BLK/g,PF/g,PTS/g
10,Raptors 905,43.5 (4),91.625 (6),0.475 (5),12.75 (14),36.875 (16),0.346 (13),30.75 (4),54.75 (7),0.562 (4),8.375 (29),13.0 (22),0.644 (31),27.375 (2),12.25 (4),7.25 (6),16.125 (31),113.25 (11)


In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-team'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Create per-game columns
    for col in per_game_columns:
        if col in team_stats.columns:  # Ensure the column exists
            team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg', 'G', 'MP']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Exclude the "League Average" row
    team_stats = team_stats[team_stats['Team'] != 'League Average'].reset_index(drop=True)
    
    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTM/g', 'FTA/g', 'FT%', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Filter the row for opponent 905 only
    opponent_row = team_stats.loc[team_stats['Team'] == 'Greensboro Swarm']
    
    # Calculate ranks and append ranks to numeric columns for display
    invert_cols = ['PF/g']  # Specify columns where ranking should be ascending
    df_with_ranks = team_stats.copy()
    for col in numeric_cols:
        # Calculate ranks (invert order for specified columns)
        if col in invert_cols:
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
        else:
            ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        
        # Append ranks to column values for display
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
    
    # Apply color coding logic to opponent 905 row
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            if col in invert_cols:
                # Invert normalization for specified columns
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            else:
                # Normal normalization
                norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Style the opponent 905 row
    styled_df = df_with_ranks.loc[df_with_ranks['Team'] == 'Greensboro Swarm'].style.apply(
        color_cells, subset=numeric_cols, axis=1
    ).format(precision=3)

else:
    print("Table with id 'team-stats-sc-team' not found.")

# Quit the driver
driver.quit()

styled_df

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTM/g,FTA/g,FT%,AST/g,STL/g,BLK/g,PF/g,PTS/g
18,Greensboro Swarm,39.25 (18),87.125 (21),0.451 (15),9.0 (30),27.125 (31),0.332 (17),30.25 (6),60.0 (2),0.504 (28),14.125 (2),18.75 (2),0.753 (15),15.25 (31),10.125 (7),4.0 (27),3.625 (1),109.125 (19)


In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-team'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Create per-game columns
    for col in per_game_columns:
        if col in team_stats.columns:  # Ensure the column exists
            team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Define the desired column order
    desired_order = [
        'Team', 'G', 'MP', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTM/g', 'FTA/g', 'FT%', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)

else:
    print("Table with id 'team-stats-sc-team' not found.")

# Quit the driver
driver.quit()

team_stats

,Team,G,MP,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTM/g,FTA/g,FT%,AST/g,STL/g,BLK/g,PF/g,PTS/g
0,Valley Suns,8,1945,44.625,91.375,0.488,18.750,44.000,0.426,25.875,47.375,0.546,10.375,14.000,0.741,25.750,8.250,4.125,17.375,125.125
1,Westchester Knicks,8,1920,46.500,98.250,0.473,11.000,34.250,0.321,35.500,64.000,0.555,9.500,14.375,0.661,25.750,10.000,8.000,12.250,120.125
2,San Diego Clippers,8,1945,41.500,91.500,0.454,12.625,36.125,0.349,28.875,55.375,0.521,14.125,20.375,0.693,22.875,12.625,3.750,4.250,118.625
3,Rip City Remix,8,1920,42.750,94.750,0.451,14.250,45.000,0.317,28.500,49.750,0.573,10.625,13.625,0.780,20.875,8.500,6.250,14.500,117.000
4,Sioux Falls Skyforce,9,2185,44.000,87.444,0.503,12.000,34.222,0.351,32.000,53.222,0.601,10.000,12.889,0.776,29.333,10.333,5.222,9.889,116.778
5,Motor City Cruise,9,2210,43.111,89.889,0.480,13.222,35.556,0.372,29.889,54.333,0.550,9.444,12.000,0.787,25.667,10.111,4.889,5.222,115.556
6,Stockton Kings,8,1920,41.375,87.500,0.473,15.625,38.125,0.410,25.750,49.375,0.522,10.125,13.500,0.750,26.250,7.125,6.000,12.875,115.125
7,Salt Lake City Stars,8,1920,41.375,91.500,0.452,11.250,35.000,0.321,30.125,56.500,0.533,12.250,15.250,0.803,23.000,10.375,3.750,9.625,114.750
8,Iowa Wolves,10,2425,42.200,87.600,0.482,9.600,28.500,0.337,32.600,59.100,0.552,11.900,16.100,0.739,25.100,10.100,5.900,13.600,114.400
9,Long Island Nets,9,2160,41.333,92.667,0.446,10.889,34.778,0.313,30.444,57.889,0.526,11.889,15.222,0.781,24.556,8.444,5.333,14.667,113.778


# Standard Stats Against

In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-opponent'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Calculate per-game statistics before dropping 'G'
    if 'G' in team_stats.columns:
        for col in per_game_columns:
            if col in team_stats.columns:  # Ensure the column exists
                team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg', 'FTM/g', 'FT%']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Drop the 'G' and 'MP' columns after calculations
    team_stats.drop(columns=['G', 'MP'], inplace=True, errors='ignore')

    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTA/g', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Filter rows for Raptors 905 and League Average
    filtered_teams = team_stats.loc[team_stats['Team'].isin(['Raptors 905', 'League Average'])]
    
else:
    print("Table with id 'team-stats-sc-team' not found.")

# Quit the driver
driver.quit()

filtered_teams

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTA/g,AST/g,STL/g,BLK/g,PF/g,PTS/g
28,Raptors 905,41.875,89.375,0.469,13.875,36.250,0.383,28.0,53.125,0.527,18.125,25.000,8.0,5.875,7.375,119.000
31,League Average,39.444,87.000,0.453,12.444,36.889,0.339,27.0,50.222,0.537,13.889,22.889,9.0,5.556,10.111,108.444


In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-opponent'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Create per-game columns
    for col in per_game_columns:
        if col in team_stats.columns:  # Ensure the column exists
            team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg', 'G', 'MP']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Drop 'FTM/g' and 'FT%' columns
    team_stats.drop(columns=['FTM/g', 'FT%'], inplace=True, errors='ignore')

    # Exclude the "League Average" row
    team_stats = team_stats[team_stats['Team'] != 'League Average'].reset_index(drop=True)
    
    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTA/g', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Calculate ranks and append ranks to numeric columns for display
    invert_cols = ['PF/g']  # PF/g will be descending, all others ascending
    df_with_ranks = team_stats.copy()
    for col in numeric_cols:
        # Calculate ranks (invert order for specified columns)
        if col in invert_cols:
            ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        else:
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
        
        # Append ranks to column values for display
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
    
    # Color coding logic
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            if col in invert_cols:
                # Invert normalization for PF/g (descending order)
                norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            else:
                # Normal normalization for ascending order
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply the coloring logic to the DataFrame with ranks
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

else:
    print("Table with id 'team-stats-sc-opponent' not found.")

# Quit the driver
driver.quit()

styled_df

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTA/g,AST/g,STL/g,BLK/g,PF/g,PTS/g
0,College Park Skyhawks,36.667 (3),87.111 (9),0.421 (2),9.333 (1),34.0 (6),0.275 (1),27.333 (15),53.111 (19),0.515 (6),15.111 (21),20.889 (4),9.333 (19),4.444 (6),13.444 (7),100.667 (1)
1,Greensboro Swarm,35.625 (1),81.375 (2),0.438 (10),11.375 (9),36.125 (13),0.315 (8),24.25 (4),45.25 (5),0.536 (17),15.125 (22),18.375 (1),7.75 (4),6.0 (18),20.5 (1),101.375 (2)
2,Birmingham Squadron,36.889 (4),87.889 (13),0.42 (1),14.667 (25),40.889 (23),0.359 (24),22.222 (2),47.0 (9),0.473 (1),10.778 (1),22.111 (9),10.0 (22),2.556 (1),8.556 (21),103.333 (3)
3,Iowa Wolves,38.0 (5),86.6 (8),0.439 (11),10.3 (4),34.0 (6),0.303 (4),27.7 (17),52.6 (16),0.527 (10),14.5 (17),24.4 (22),10.1 (24),6.2 (23),10.0 (14),104.5 (4)
4,Wisconsin Herd,38.125 (6),87.5 (11),0.436 (8),10.0 (2),31.375 (3),0.319 (10),28.125 (19),56.125 (26),0.501 (3),15.875 (27),22.5 (11),8.25 (9),6.75 (26),11.75 (10),104.625 (5)
5,Memphis Hustle,36.2 (2),83.4 (4),0.434 (7),15.5 (30),43.5 (28),0.356 (23),20.7 (1),39.9 (1),0.519 (9),13.3 (11),23.8 (17),7.9 (5),7.3 (28),15.2 (3),105.4 (6)
6,Austin Spurs,39.111 (11),81.111 (1),0.482 (29),13.556 (19),34.444 (8),0.394 (29),25.556 (8),46.667 (7),0.548 (23),11.556 (3),19.222 (2),8.0 (6),5.889 (17),11.111 (11),106.111 (7)
7,Sioux Falls Skyforce,39.444 (12),87.111 (9),0.453 (15),10.0 (2),32.556 (5),0.307 (5),29.444 (25),54.556 (24),0.54 (19),13.556 (14),22.778 (14),8.889 (17),5.333 (11),9.556 (15),106.222 (8)
8,Mexico City Capitanes,38.444 (8),89.0 (16),0.432 (6),12.444 (15),42.889 (25),0.29 (2),26.0 (11),46.111 (6),0.564 (25),13.778 (15),21.444 (6),12.0 (30),5.556 (12),7.333 (25),106.667 (9)
9,Oklahoma City Blue,39.444 (12),90.556 (24),0.436 (8),14.0 (22),45.444 (31),0.308 (6),25.444 (7),45.111 (4),0.564 (25),12.0 (5),22.444 (10),10.0 (22),6.778 (27),7.111 (27),107.0 (10)


In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-opponent'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Create per-game columns
    for col in per_game_columns:
        if col in team_stats.columns:  # Ensure the column exists
            team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg', 'G', 'MP']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Drop 'FTM/g' and 'FT%' columns
    team_stats.drop(columns=['FTM/g', 'FT%'], inplace=True, errors='ignore')

    # Exclude the "League Average" row
    team_stats = team_stats[team_stats['Team'] != 'League Average'].reset_index(drop=True)
    
    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTA/g', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Calculate ranks and append ranks to numeric columns for display
    invert_cols = ['PF/g']  # PF/g will be descending, all others ascending
    df_with_ranks = team_stats.copy()
    for col in numeric_cols:
        # Calculate ranks (invert order for specified columns)
        if col in invert_cols:
            ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        else:
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
        
        # Append ranks to column values for display
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
    
    # Apply color coding logic to Raptors 905 row
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            if col in invert_cols:
                # Invert normalization for PF/g (descending order)
                norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            else:
                # Normal normalization for ascending order
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Filter for Raptors 905 only
    raptors_row = df_with_ranks[df_with_ranks['Team'] == 'Raptors 905']
    
    # Style the Raptors 905 row
    styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

else:
    print("Table with id 'team-stats-sc-opponent' not found.")

# Quit the driver
driver.quit()

styled_df

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTA/g,AST/g,STL/g,BLK/g,PF/g,PTS/g
28,Raptors 905,41.875 (24),89.375 (18),0.469 (22),13.875 (20),36.25 (14),0.383 (28),28.0 (18),53.125 (20),0.527 (10),18.125 (31),25.0 (25),8.0 (6),5.875 (16),7.375 (24),119.0 (29)


In [38]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-opponent'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    
    # Remove the 'Rk' column
    if 'Rk' in team_stats.columns:
        team_stats.drop(columns=['Rk'], inplace=True)
    
    # List of columns to transform to per-game
    per_game_columns = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 
                        'FT', 'FTA', 'AST', 'STL', 'BLK', 'PF', 'PTS']
    
    # Create per-game columns
    for col in per_game_columns:
        if col in team_stats.columns:  # Ensure the column exists
            team_stats[f'{col}/g'] = team_stats[col] / team_stats['G']
    
    # Drop specified columns and their per-game equivalents
    drop_columns = ['TOV', 'ORB', 'DRB', 'TRB', 'Pace', 'ORtg', 'G', 'MP']
    team_stats.drop(columns=[col for col in drop_columns if col in team_stats.columns], inplace=True)
    team_stats.drop(columns=[f'{col}/g' for col in drop_columns if f'{col}/g' in team_stats.columns], inplace=True)
    
    # Rename specific columns
    rename_mapping = {
        'FG/g': 'FGM/g',
        '3P/g': '3PM/g',
        '2P/g': '2PM/g',
        'FT/g': 'FTM/g'
    }
    team_stats.rename(columns=rename_mapping, inplace=True)
    
    # Drop 'FTM/g' and 'FT%' columns
    team_stats.drop(columns=['FTM/g', 'FT%'], inplace=True, errors='ignore')

    # Exclude the "League Average" row
    team_stats = team_stats[team_stats['Team'] != 'League Average'].reset_index(drop=True)
    
    # Define the desired column order
    desired_order = [
        'Team', 'FGM/g', 'FGA/g', 'FG%', '3PM/g', '3PA/g', '3P%', '2PM/g',
        '2PA/g', '2P%', 'FTA/g', 'AST/g', 'STL/g', 'BLK/g', 'PF/g', 'PTS/g'
    ]
    
    # Reorder columns (ignoring missing ones in the DataFrame)
    team_stats = team_stats[[col for col in desired_order if col in team_stats.columns]]

    # Round all numeric columns to three decimal points
    numeric_cols = team_stats.select_dtypes(include='number').columns
    team_stats[numeric_cols] = team_stats[numeric_cols].round(3)
    
    # Calculate ranks and append ranks to numeric columns for display
    invert_cols = ['PF/g']  # PF/g will be descending, all others ascending
    df_with_ranks = team_stats.copy()
    for col in numeric_cols:
        # Calculate ranks (invert order for specified columns)
        if col in invert_cols:
            ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        else:
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
        
        # Append ranks to column values for display
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
    
    # Apply color coding logic to opponent 905 row
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            if col in invert_cols:
                # Invert normalization for PF/g (descending order)
                norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            else:
                # Normal normalization for ascending order
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Filter for opponent 905 only
    opponent_row = df_with_ranks[df_with_ranks['Team'] == 'Greensboro Swarm']
    
    # Style the opponent 905 row
    styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

else:
    print("Table with id 'team-stats-sc-opponent' not found.")

# Quit the driver
driver.quit()

styled_df

,Team,FGM/g,FGA/g,FG%,3PM/g,3PA/g,3P%,2PM/g,2PA/g,2P%,FTA/g,AST/g,STL/g,BLK/g,PF/g,PTS/g
1,Greensboro Swarm,35.625 (1),81.375 (2),0.438 (10),11.375 (9),36.125 (13),0.315 (8),24.25 (4),45.25 (5),0.536 (17),15.125 (22),18.375 (1),7.75 (4),6.0 (18),20.5 (1),101.375 (2)


In [10]:
team_stats.columns

Index(['Rk', 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
       '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'Pace', 'ORtg'],
      dtype='object')

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=service, options=options)

# Load the webpage
url = 'https://www.basketball-reference.com/gleague/years/2025.html'
driver.get(url)

# Allow some time for JavaScript to load
time.sleep(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'team-stats-sc-team'})

# Convert to pandas DataFrame
if table:
    team_stats = pd.read_html(str(table))[0]
    print(team_stats)  # Display the DataFrame
else:
    print("Table with id 'team-stats-sc-team' not found.")

# Quit the driver
driver.quit()


      Rk                      Team   G    MP   FG  FGA    FG%   3P  3PA  \
0    1.0               Valley Suns   8  1945  357  731  0.488  150  352   
1    2.0        Westchester Knicks   8  1920  372  786  0.473   88  274   
2    3.0        San Diego Clippers   8  1945  332  732  0.454  101  289   
3    4.0            Rip City Remix   8  1920  342  758  0.451  114  360   
4    5.0      Sioux Falls Skyforce   9  2185  396  787  0.503  108  308   
5    6.0         Motor City Cruise   9  2210  388  809  0.480  119  320   
6    7.0            Stockton Kings   8  1920  331  700  0.473  125  305   
7    8.0      Salt Lake City Stars   8  1920  331  732  0.452   90  280   
8    9.0               Iowa Wolves  10  2425  422  876  0.482   96  285   
9   10.0          Long Island Nets   9  2160  372  834  0.446   98  313   
10  11.0               Raptors 905   8  1920  348  733  0.475  102  295   
11  12.0              Austin Spurs   9  2185  370  841  0.440  117  390   
12  13.0             Main

# G League Website

## Traditional Stats

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/traditional/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'WIN% RANK', 'MIN RANK', 'PTS RANK',
    'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
    'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
    'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'BLKA RANK',
    'PF RANK', 'PFD RANK', '+/- RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['PF']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
0,Iowa Wolves,111.9 (13),41.2 (11),88.3 (16),46.7 (8),9.8 (30),30.3 (29),32.5 (22),11.6 (5),15.8 (5),73.3 (23),13.5 (9),32.9 (17),46.4 (12),24.3 (16),17.5 (10),10.3 (5),5.7 (17),6.5 (10),21.6 (26),22.5 (4),5.8 (5)
1,Austin Spurs,112.8 (11),40.4 (14),92.8 (4),43.5 (24),13.7 (9),43.7 (6),31.3 (28),10.8 (13),14.8 (7),73.4 (22),14.9 (5),30.5 (27),45.4 (16),22.8 (22),14.1 (29),12.5 (1),5.8 (15),6.8 (6),16.8 (1),20.2 (14),7.3 (2)
2,Greensboro Swarm,107.3 (24),38.1 (25),85.4 (28),44.6 (18),9.2 (31),26.3 (31),34.8 (10),13.8 (1),18.3 (1),75.5 (12),14.8 (6),33.8 (10),48.6 (5),19.0 (31),15.8 (22),7.9 (26),4.5 (27),6.6 (7),21.1 (19),24.4 (1),2.8 (10)
3,Oklahoma City Blue,110.0 (18),37.7 (28),86.6 (24),43.5 (24),17.2 (2),45.6 (2),37.7 (4),10.9 (10),13.5 (18),80.9 (1),12.0 (16),34.3 (8),46.3 (13),24.7 (14),17.8 (8),8.8 (15),4.0 (30),6.1 (14),18.3 (5),19.1 (24),4.3 (7)
4,Valley Suns,122.7 (1),44.3 (2),92.1 (7),48.1 (3),17.9 (1),44.5 (3),40.3 (1),10.1 (16),13.5 (18),74.5 (17),9.2 (30),33.1 (15),42.3 (25),28.2 (1),13.9 (30),8.0 (25),4.8 (25),4.4 (29),21.5 (25),21.5 (8),5.9 (4)
5,Stockton Kings,115.8 (6),41.3 (10),88.6 (15),46.6 (9),15.1 (4),38.2 (11),39.5 (2),10.8 (13),14.4 (12),75.0 (15),12.8 (13),32.4 (19),45.2 (17),26.2 (5),15.8 (22),7.6 (27),6.0 (14),3.4 (31),19.8 (15),20.1 (15),2.1 (12)
6,Capital City Go-Go,111.5 (14),39.2 (20),80.5 (31),48.6 (2),10.8 (27),29.3 (30),37.0 (5),13.3 (2),16.5 (4),80.8 (2),10.6 (27),35.5 (4),46.1 (15),21.9 (27),21.2 (1),6.6 (30),6.8 (7),6.4 (11),22.2 (28),23.5 (2),3.4 (8)
7,College Park Skyhawks,108.7 (21),40.0 (17),86.3 (26),46.4 (11),12.8 (14),37.7 (15),33.8 (17),9.8 (20),13.2 (22),74.1 (19),11.5 (19),35.1 (6),46.6 (10),23.8 (19),18.0 (7),7.1 (28),5.3 (19),4.9 (27),22.0 (27),20.0 (16),4.7 (6)
8,Sioux Falls Skyforce,114.0 (9),43.1 (3),88.3 (16),48.8 (1),11.4 (25),35.1 (24),32.5 (22),9.8 (20),12.8 (26),77.1 (8),12.0 (16),35.6 (3),47.6 (6),28.1 (2),17.7 (9),10.0 (6),5.5 (18),5.7 (18),19.0 (9),19.2 (23),9.2 (1)
9,Grand Rapids Gold,107.7 (23),40.7 (13),87.5 (21),46.5 (10),12.5 (17),36.5 (19),34.4 (14),8.3 (28),10.6 (30),77.8 (7),11.9 (18),29.7 (29),41.6 (27),22.5 (26),14.7 (26),8.5 (20),4.9 (24),5.1 (25),19.2 (11),15.7 (31),-0.6 (17)


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/traditional/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'WIN% RANK', 'MIN RANK', 'PTS RANK',
    'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
    'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
    'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'BLKA RANK',
    'PF RANK', 'PFD RANK', '+/- RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['PF']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Raptors 905']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
26,Raptors 905,112.4 (12),43.0 (4),90.2 (12),47.7 (5),12.5 (17),35.8 (23),34.9 (9),8.3 (28),13.4 (20),62.1 (31),11.3 (23),29.9 (28),41.3 (28),26.6 (4),17.3 (14),10.7 (4),7.0 (6),5.7 (18),25.9 (31),21.8 (7),-5.3 (26)


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/traditional/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'WIN% RANK', 'MIN RANK', 'PTS RANK',
    'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
    'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
    'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'BLKA RANK',
    'PF RANK', 'PFD RANK', '+/- RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['PF']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Delaware Blue Coats']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
16,Delaware Blue Coats,110.3 (17),39.0 (21),87.5 (21),44.6 (18),12.2 (19),36.7 (17),33.3 (18),12.8 (4),16.8 (3),76.1 (10),11.3 (23),31.8 (21),43.1 (23),22.7 (24),16.6 (20),8.4 (21),5.3 (19),5.5 (21),17.8 (3),22.3 (6),-3.5 (22)


## Advanced Stats

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/advanced/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK',
       'MIN RANK', 'OffRtg RANK', 'DefRtg RANK', 'NetRtg RANK', 'AST% RANK',
       'AST/TO RANK', 'AST Ratio RANK', 'OREB% RANK', 'DREB% RANK',
       'REB% RANK', 'TO Ratio RANK', 'eFG% RANK', 'TS% RANK', 'PACE RANK',
       'PIE RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['DefRtg', 'TOV%']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,OffRtg,DefRtg,NetRtg,AST%,AST/TO,AST Ratio,OREB%,DREB%,REB%,TOV%,eFG%,TS%,PACE,PIE
0,Iowa Wolves,109.5 (15),103.8 (2),5.6 (4),59.0 (18),1.39 (17),16.9 (19),34.6 (7),70.1 (8),52.3 (6),17.2 (18),52.3 (15),58.7 (11),101.63 (13),52.8 (6)
1,Austin Spurs,113.6 (5),106.1 (6),7.5 (2),56.3 (26),1.62 (7),16.1 (25),35.5 (4),69.9 (9),51.1 (9),14.2 (5),50.9 (24),56.8 (24),98.85 (23),53.5 (4)
2,Capital City Go-Go,109.3 (16),106.3 (7),3.0 (9),56.0 (28),1.03 (31),15.9 (27),32.0 (12),67.1 (20),51.7 (7),20.8 (31),55.3 (3),63.5 (1),101.85 (12),53.8 (3)
3,Greensboro Swarm,110.2 (13),107.6 (14),2.5 (10),49.9 (31),1.2 (29),14.1 (31),36.9 (2),72.4 (5),54.5 (2),16.3 (13),50.0 (27),57.4 (21),96.83 (29),51.8 (8)
4,Oklahoma City Blue,111.1 (9),106.5 (8),4.7 (6),65.5 (4),1.39 (17),17.7 (9),31.4 (15),68.7 (12),50.6 (13),17.9 (24),53.4 (11),59.4 (7),99.17 (22),52.0 (7)
5,College Park Skyhawks,108.5 (17),104.2 (4),4.3 (7),59.4 (17),1.32 (23),17.1 (16),31.4 (15),72.9 (2),52.8 (5),18.0 (25),53.8 (7),59.0 (9),100.0 (20),51.8 (8)
6,Sioux Falls Skyforce,111.3 (8),102.8 (1),8.5 (1),65.2 (5),1.59 (8),19.3 (2),31.7 (14),73.1 (1),53.1 (4),17.2 (18),55.3 (3),60.7 (5),101.59 (14),56.4 (1)
7,Valley Suns,116.6 (1),111.4 (23),5.1 (5),63.7 (6),2.03 (1),19.4 (1),23.6 (31),65.7 (26),45.6 (30),13.2 (1),57.8 (1),62.6 (2),103.81 (8),52.9 (5)
8,Ciudad de Mexico Capitanes,102.9 (29),104.1 (3),-1.2 (20),57.7 (22),1.07 (30),15.6 (28),25.3 (30),66.2 (25),46.7 (28),19.3 (29),52.2 (18),58.4 (14),100.17 (17),47.8 (25)
9,Grand Rapids Gold,111.7 (7),112.3 (26),-0.7 (19),55.1 (29),1.52 (12),16.8 (20),31.1 (18),68.7 (12),49.4 (19),15.3 (7),53.7 (8),58.4 (14),95.85 (30),48.4 (22)


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/advanced/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK',
       'MIN RANK', 'OffRtg RANK', 'DefRtg RANK', 'NetRtg RANK', 'AST% RANK',
       'AST/TO RANK', 'AST Ratio RANK', 'OREB% RANK', 'DREB% RANK',
       'REB% RANK', 'TO Ratio RANK', 'eFG% RANK', 'TS% RANK', 'PACE RANK',
       'PIE RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['DefRtg', 'TOV%']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Raptors 905']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,OffRtg,DefRtg,NetRtg,AST%,AST/TO,AST Ratio,OREB%,DREB%,REB%,TOV%,eFG%,TS%,PACE,PIE
26,Raptors 905,108.4 (19),112.9 (27),-4.6 (24),61.8 (14),1.53 (11),18.3 (5),29.5 (23),64.5 (29),46.8 (27),16.7 (15),54.6 (6),58.5 (12),104.0 (7),48.0 (24)


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/advanced/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK',
       'MIN RANK', 'OffRtg RANK', 'DefRtg RANK', 'NetRtg RANK', 'AST% RANK',
       'AST/TO RANK', 'AST Ratio RANK', 'OREB% RANK', 'DREB% RANK',
       'REB% RANK', 'TO Ratio RANK', 'eFG% RANK', 'TS% RANK', 'PACE RANK',
       'PIE RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['DefRtg', 'TOV%']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
opponent_row = df_with_ranks[df_with_ranks['TEAM'] == 'Delaware Blue Coats']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,OffRtg,DefRtg,NetRtg,AST%,AST/TO,AST Ratio,OREB%,DREB%,REB%,TOV%,eFG%,TS%,PACE,PIE
11,Delaware Blue Coats,108.5 (17),111.5 (24),-3.1 (22),58.2 (21),1.37 (21),16.2 (24),29.4 (24),65.7 (26),47.6 (26),16.3 (13),51.6 (20),58.2 (16),101.88 (11),48.4 (22)


In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/four-factors/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'eFG% RANK', 'TO Ratio RANK',
    'OREB% RANK', 'Opp FTA Rate RANK', 'Opp To Ratio RANK', 'WIN% RANK', 'FTA Rate RANK', 'Opp eFG% RANK', 'Opp OREB% RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['TOV%', 'Opp eFG%', 'Opp FTA Rate', 'Opp OREB%']  # Columns with descending rank
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,eFG%,FTA Rate,TOV%,OREB%,Opp eFG%,Opp FTA Rate,Opp TOV%,Opp OREB%
0,Iowa Wolves,52.3 (15),0.179 (5),17.2 (18),34.6 (7),51.3 (9),0.163 (19),18.4 (5),29.9 (8)
1,Austin Spurs,50.9 (24),0.159 (11),14.2 (5),35.5 (4),54.7 (27),0.141 (6),20.3 (2),30.1 (9)
2,Greensboro Swarm,50.0 (27),0.215 (1),16.3 (13),36.9 (2),52.2 (15),0.168 (22),15.5 (22),27.6 (5)
3,Oklahoma City Blue,53.4 (11),0.156 (12),17.9 (24),31.4 (15),51.8 (12),0.127 (2),16.7 (15),31.3 (12)
4,Valley Suns,57.8 (1),0.147 (22),13.2 (1),23.6 (31),51.0 (8),0.171 (24),15.1 (25),34.3 (26)
5,Stockton Kings,55.1 (5),0.163 (10),15.8 (9),33.7 (9),53.2 (22),0.148 (10),14.6 (29),33.2 (22)
6,Capital City Go-Go,55.3 (3),0.204 (2),20.8 (31),32.0 (12),48.8 (1),0.172 (25),14.8 (27),32.9 (20)
7,College Park Skyhawks,53.8 (7),0.153 (16),18.0 (25),31.4 (15),49.3 (2),0.178 (26),14.7 (28),27.1 (2)
8,Sioux Falls Skyforce,55.3 (3),0.144 (25),17.2 (18),31.7 (14),50.2 (5),0.156 (12),16.9 (14),26.9 (1)
9,Grand Rapids Gold,53.7 (8),0.121 (30),15.3 (7),31.1 (18),55.7 (29),0.144 (9),17.0 (12),31.3 (12)


In [52]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/four-factors/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'eFG% RANK', 'TO Ratio RANK',
    'OREB% RANK', 'Opp FTA Rate RANK', 'Opp To Ratio RANK', 'WIN% RANK', 'FTA Rate RANK', 'Opp eFG% RANK', 'Opp OREB% RANK'
]

team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Check for NaN or inf values
#print("Checking for NaN and inf values...")
#print(team_stats[numeric_cols].isna().sum())
#print((team_stats[numeric_cols] == np.inf).sum())
#print((team_stats[numeric_cols] == -np.inf).sum())

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['TOV%', 'Opp eFG%', 'Opp FTA Rate', 'Opp OREB%']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Raptors 905']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,eFG%,FTA Rate,TOV%,OREB%,Opp eFG%,Opp FTA Rate,Opp TOV%,Opp OREB%
26,Raptors 905,54.8 (6),0.146 (24),16.9 (17),29.6 (24),54.9 (27),0.21 (31),18.2 (7),35.2 (26)


In [53]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/four-factors/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'eFG% RANK', 'TO Ratio RANK',
    'OREB% RANK', 'Opp FTA Rate RANK', 'Opp To Ratio RANK', 'WIN% RANK', 'FTA Rate RANK', 'Opp eFG% RANK', 'Opp OREB% RANK'
]

team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Check for NaN or inf values
#print("Checking for NaN and inf values...")
#print(team_stats[numeric_cols].isna().sum())
#print((team_stats[numeric_cols] == np.inf).sum())
#print((team_stats[numeric_cols] == -np.inf).sum())

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['TOV%', 'Opp eFG%', 'Opp FTA Rate', 'Opp OREB%']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
opponents_row = df_with_ranks[df_with_ranks['TEAM'] == 'Westchester Knicks']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = opponents_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,eFG%,FTA Rate,TOV%,OREB%,Opp eFG%,Opp FTA Rate,Opp TOV%,Opp OREB%
11,Westchester Knicks,53.6 (8),0.151 (16),16.2 (10),39.4 (1),51.2 (10),0.164 (20),16.5 (17),32.1 (18)


In [30]:
print(team_stats.columns.tolist())

['TEAM', 'eFG%', 'FTA Rate', 'TOV%', 'OREB%', 'Opp eFG%', 'Opp FTA\xa0Rate', 'Opp TOV%', 'Opp OREB%', 'Opp FTA\xa0Rate RANK', 'Opp To\xa0Ratio RANK']


In [29]:
team_stats.columns

Index(['TEAM', 'eFG%', 'FTA Rate', 'TOV%', 'OREB%', 'Opp eFG%', 'Opp FTA Rate',
       'Opp TOV%', 'Opp OREB%', 'Opp FTA Rate RANK', 'Opp To Ratio RANK'],
      dtype='object')

In [41]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/misc/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'PTS OFF TO RANK',
       '2nd PTS RANK', 'FBPs RANK', 'PITP RANK', 'Opp PTS OFF TO RANK',
       'Opp 2nd PTS RANK', 'Opp FBPs RANK', 'Opp PITP RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['Opp PTS OFF TO', 'Opp 2nd PTS', 'Opp FBPs', 'Opp PITP']  # Columns with descending rank
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS OFF TO,2nd PTS,FBPs,PITP,Opp PTS OFF TO,Opp 2nd PTS,Opp FBPs,Opp PITP
0,Iowa Wolves,22.3 (7),20.2 (2),19.8 (3),61.4 (2),19.4 (14),11.6 (2),16.7 (25),49.8 (16)
1,Greensboro Swarm,18.7 (20),16.8 (14),9.8 (29),56.0 (5),17.0 (4),12.2 (5),10.6 (3),43.8 (4)
2,Austin Spurs,24.2 (2),17.1 (11),17.1 (5),50.7 (14),18.4 (8),12.1 (4),12.1 (6),47.1 (9)
3,Capital City Go-Go,19.7 (18),17.3 (9),12.1 (23),52.2 (11),24.1 (28),17.1 (23),14.5 (17),54.6 (27)
4,College Park Skyhawks,16.6 (29),14.5 (22),15.9 (9),47.4 (20),20.2 (18),11.0 (1),9.9 (2),49.4 (14)
5,Oklahoma City Blue,20.0 (16),14.9 (17),9.7 (30),38.2 (31),20.2 (18),18.1 (25),11.6 (4),47.6 (11)
6,Sioux Falls Skyforce,20.4 (14),13.8 (26),21.2 (2),59.8 (3),16.6 (3),13.9 (7),14.1 (15),53.2 (25)
7,Stockton Kings,17.1 (27),18.7 (4),13.8 (18),44.4 (27),19.8 (16),16.4 (21),17.4 (29),52.4 (22)
8,Valley Suns,21.0 (11),11.0 (29),14.4 (16),45.2 (26),18.6 (10),22.1 (31),14.5 (17),51.8 (19)
9,Ciudad de Mexico Capitanes,22.8 (5),10.1 (31),17.1 (5),41.4 (30),23.3 (27),16.3 (20),17.9 (30),50.6 (17)


In [54]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/misc/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'PTS OFF TO RANK',
       '2nd PTS RANK', 'FBPs RANK', 'PITP RANK', 'Opp PTS OFF TO RANK',
       'Opp 2nd PTS RANK', 'Opp FBPs RANK', 'Opp PITP RANK'
]

team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Check for NaN or inf values
#print("Checking for NaN and inf values...")
#print(team_stats[numeric_cols].isna().sum())
#print((team_stats[numeric_cols] == np.inf).sum())
#print((team_stats[numeric_cols] == -np.inf).sum())

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['Opp PTS OFF TO', 'Opp 2nd PTS', 'Opp FBPs', 'Opp PITP']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Raptors 905']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS OFF TO,2nd PTS,FBPs,PITP,Opp PTS OFF TO,Opp 2nd PTS,Opp FBPs,Opp PITP
26,Raptors 905,20.7 (11),14.5 (19),14.1 (15),56.9 (4),21.2 (23),17.5 (26),13.0 (9),52.2 (21)


In [55]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/misc/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'PTS OFF TO RANK',
       '2nd PTS RANK', 'FBPs RANK', 'PITP RANK', 'Opp PTS OFF TO RANK',
       'Opp 2nd PTS RANK', 'Opp FBPs RANK', 'Opp PITP RANK'
]

team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Check for NaN or inf values
#print("Checking for NaN and inf values...")
#print(team_stats[numeric_cols].isna().sum())
#print((team_stats[numeric_cols] == np.inf).sum())
#print((team_stats[numeric_cols] == -np.inf).sum())

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['Opp PTS OFF TO', 'Opp 2nd PTS', 'Opp FBPs', 'Opp PITP']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for opponents 905 only
opponents_row = df_with_ranks[df_with_ranks['TEAM'] == 'Westchester Knicks']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the opponents 905 row
styled_df = opponents_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS OFF TO,2nd PTS,FBPs,PITP,Opp PTS OFF TO,Opp 2nd PTS,Opp FBPs,Opp PITP
19,Westchester Knicks,23.0 (6),19.0 (3),16.4 (6),67.3 (1),21.2 (23),16.2 (17),13.8 (14),55.1 (27)


In [40]:
team_stats.columns

Index(['Unnamed: 0', 'TEAM', 'GP', 'W', 'L', 'MIN', 'PTS OFF TO', '2nd PTS',
       'FBPs', 'PITP', 'Opp PTS OFF TO', 'Opp 2nd PTS', 'Opp FBPs', 'Opp PITP',
       'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'PTS OFF TO RANK',
       '2nd PTS RANK', 'FBPs RANK', 'PITP RANK', 'Opp PTS OFF TO RANK',
       'Opp 2nd PTS RANK', 'Opp FBPs RANK', 'Opp PITP RANK'],
      dtype='object')

In [39]:
print(team_stats.columns.tolist())

['Unnamed: 0', 'TEAM', 'GP', 'W', 'L', 'MIN', 'PTS OFF\xa0TO', '2nd PTS', 'FBPs', 'PITP', 'Opp\xa0PTS OFF\xa0TO', 'Opp 2nd\xa0PTS', 'Opp FBPs', 'Opp PITP', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'PTS OFF\xa0TO RANK', '2nd PTS RANK', 'FBPs RANK', 'PITP RANK', 'Opp\xa0PTS OFF\xa0TO RANK', 'Opp 2nd\xa0PTS RANK', 'Opp FBPs RANK', 'Opp PITP RANK']


In [47]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/scoring/"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK',
    'MIN RANK', '%FGA 2PT RANK', '%FGA 3PT RANK', '%PTS 2PT RANK',
    '%PTS 2PT- MR RANK', '%PTS 3PT RANK', '%PTS FBPs RANK', '%PTS FT RANK',
    '%PTS OffTO RANK', '%PTS PITP RANK', '2FGM %AST RANK',
    '2FGM %UAST RANK', '3FGM %AST RANK', '3FGM %UAST RANK', 'FGM %AST RANK',
    'FGM %UAST RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (all in descending order since no inversion is needed)
    ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        # Normal normalization for all columns (descending rank)
        norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,%FGA 2PT,%FGA 3PT,%PTS 2PT,%PTS 2PT- MR,%PTS 3PT,%PTS FBPs,%PTS FT,%PTS OffTO,%PTS PITP,2FGM %AST,2FGM %UAST,3FGM %AST,3FGM %UAST,FGM %AST,FGM %UAST
0,Iowa Wolves,67.7 (2),32.3 (30),57.0 (2),3.3 (19),25.2 (30),17.3 (3),10.5 (8),19.5 (8),53.7 (2),51.8 (6),48.2 (26),79.2 (27),20.8 (5),58.1 (19),41.9 (13)
1,Greensboro Swarm,68.8 (1),31.2 (31),54.9 (4),4.2 (16),25.1 (31),8.9 (29),12.9 (2),16.9 (21),50.7 (5),33.7 (31),66.3 (1),84.3 (16),15.7 (16),45.5 (31),54.5 (1)
2,Austin Spurs,53.9 (24),46.1 (8),49.6 (17),4.9 (15),34.4 (15),15.1 (7),9.4 (14),21.4 (4),44.7 (18),42.7 (27),57.3 (5),82.1 (21),17.9 (11),55.1 (27),44.9 (4)
3,Capital City Go-Go,61.9 (8),38.1 (24),48.5 (21),1.4 (30),30.0 (25),10.9 (23),13.2 (1),17.7 (17),47.0 (11),46.5 (19),53.5 (12),80.2 (24),19.8 (8),56.3 (22),43.7 (10)
4,College Park Skyhawks,55.8 (20),44.2 (12),50.3 (16),6.0 (7),34.8 (13),14.9 (8),9.2 (17),15.5 (28),44.3 (20),46.5 (19),53.5 (12),79.0 (28),21.0 (4),56.7 (20),43.3 (12)
5,Oklahoma City Blue,48.1 (30),51.9 (2),37.0 (31),2.2 (26),45.5 (1),8.8 (30),11.0 (6),18.2 (13),34.7 (31),46.4 (21),53.6 (11),90.7 (6),9.3 (25),66.4 (3),33.6 (29)
6,Sioux Falls Skyforce,59.7 (13),40.3 (19),54.9 (4),3.1 (21),30.9 (22),18.4 (2),8.5 (23),17.7 (17),51.8 (3),54.3 (3),45.7 (29),86.6 (11),13.4 (21),63.1 (6),36.9 (26)
7,Stockton Kings,56.5 (18),43.5 (14),44.3 (27),5.9 (8),40.2 (5),11.9 (19),9.4 (14),14.8 (29),38.3 (29),49.4 (12),50.6 (20),78.6 (29),21.4 (3),60.4 (12),39.6 (20)
8,Valley Suns,51.2 (28),48.8 (4),42.5 (29),5.7 (9),43.8 (3),11.7 (20),8.6 (22),17.1 (20),36.8 (30),52.1 (5),47.9 (27),79.9 (25),20.1 (6),63.4 (5),36.6 (27)
9,Ciudad de Mexico Capitanes,54.5 (22),45.5 (10),44.9 (26),5.4 (12),37.6 (7),16.3 (4),11.2 (5),21.8 (3),39.6 (27),43.0 (25),57.0 (6),77.9 (30),22.1 (2),55.5 (26),44.5 (6)


In [56]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/scoring/"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK',
    'MIN RANK', '%FGA 2PT RANK', '%FGA 3PT RANK', '%PTS 2PT RANK',
    '%PTS 2PT- MR RANK', '%PTS 3PT RANK', '%PTS FBPs RANK', '%PTS FT RANK',
    '%PTS OffTO RANK', '%PTS PITP RANK', '2FGM %AST RANK',
    '2FGM %UAST RANK', '3FGM %AST RANK', '3FGM %UAST RANK', 'FGM %AST RANK',
    'FGM %UAST RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (all in descending order)
    ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Raptors 905']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        # Normal normalization for all columns
        norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,%FGA 2PT,%FGA 3PT,%PTS 2PT,%PTS 2PT- MR,%PTS 3PT,%PTS FBPs,%PTS FT,%PTS OffTO,%PTS PITP,2FGM %AST,2FGM %UAST,3FGM %AST,3FGM %UAST,FGM %AST,FGM %UAST
26,Raptors 905,60.3 (12),39.7 (20),53.8 (7),3.1 (20),34.3 (17),12.6 (16),7.3 (31),18.5 (14),50.7 (4),52.1 (6),47.9 (26),88.7 (9),11.3 (23),63.0 (6),37.0 (26)


In [57]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/scoring/"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK',
    'MIN RANK', '%FGA 2PT RANK', '%FGA 3PT RANK', '%PTS 2PT RANK',
    '%PTS 2PT- MR RANK', '%PTS 3PT RANK', '%PTS FBPs RANK', '%PTS FT RANK',
    '%PTS OffTO RANK', '%PTS PITP RANK', '2FGM %AST RANK',
    '2FGM %UAST RANK', '3FGM %AST RANK', '3FGM %UAST RANK', 'FGM %AST RANK',
    'FGM %UAST RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (all in descending order)
    ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for opponent 905 only
opponent_row = df_with_ranks[df_with_ranks['TEAM'] == 'Westchester Knicks']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        # Normal normalization for all columns
        norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the opponent 905 row
styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,%FGA 2PT,%FGA 3PT,%PTS 2PT,%PTS 2PT- MR,%PTS 3PT,%PTS FBPs,%PTS FT,%PTS OffTO,%PTS PITP,2FGM %AST,2FGM %UAST,3FGM %AST,3FGM %UAST,FGM %AST,FGM %UAST
19,Westchester Knicks,64.7 (3),35.3 (29),58.2 (1),2.6 (25),28.3 (28),13.6 (12),8.1 (27),19.0 (10),55.6 (1),47.0 (18),53.0 (14),86.4 (10),13.6 (22),56.7 (22),43.3 (8)


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/opponent/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'Opp FGM RANK',
       'Opp FGA RANK', 'Opp FG% RANK', 'Opp 3PM RANK', 'Opp 3PA RANK',
       'Opp 3P% RANK', 'Opp FTM RANK', 'Opp FTA RANK', 'Opp FT% RANK',
       'Opp OREB RANK', 'Opp DREB RANK', 'Opp REB RANK', 'Opp AST RANK',
       'Opp TOV RANK', 'Opp STL RANK', 'Opp BLK RANK', 'Opp BLKA RANK',
       'Opp PF RANK', 'Opp PFD RANK', 'Opp PTS RANK', '+/- RANK', 'Opp FTM', 'Opp FT%', 'Opp DREB', 'Opp PFD' 
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['Opp PF', 'Opp TOV']  # Columns with descending rank
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,Opp FGM,Opp FGA,Opp FG%,Opp 3PM,Opp 3PA,Opp 3P%,Opp FTA,Opp OREB,Opp REB,Opp AST,Opp TOV,Opp STL,Opp BLK,Opp BLKA,Opp PF,Opp PTS,+/-
0,Iowa Wolves,38.0 (5),86.7 (8),43.8 (11),10.3 (4),34.0 (5),30.3 (6),14.5 (18),11.0 (9),41.2 (2),23.1 (11),17.7 (11),9.8 (24),6.6 (24),6.7 (23),22.8 (4),104.5 (5),-9.9 (2)
1,Greensboro Swarm,36.8 (3),83.5 (3),44.1 (13),12.2 (11),36.0 (12),33.9 (16),14.9 (21),10.0 (1),39.6 (1),22.8 (8),16.4 (18),7.5 (4),6.2 (21),4.8 (8),24.7 (2),103.1 (2),-5.5 (7)
2,Sioux Falls Skyforce,38.3 (7),87.2 (12),43.9 (12),9.9 (2),34.6 (7),28.6 (2),13.9 (14),10.5 (5),42.1 (6),21.6 (4),17.3 (15),8.6 (15),5.7 (14),5.0 (10),19.5 (22),104.0 (3),-10.2 (1)
3,Valley Suns,42.0 (25),94.3 (29),44.6 (16),12.3 (12),37.4 (19),32.8 (9),16.2 (27),14.9 (27),51.0 (31),22.9 (9),15.9 (22),7.4 (3),4.4 (3),4.6 (6),21.5 (7),116.6 (29),-6.1 (4)
4,Austin Spurs,39.1 (11),82.4 (1),47.4 (26),13.6 (22),34.6 (7),39.2 (31),11.6 (2),10.7 (7),43.2 (9),20.6 (1),20.6 (2),7.6 (6),6.9 (26),5.6 (12),19.7 (19),106.1 (8),-7.2 (3)
5,Capital City Go-Go,40.1 (17),93.6 (28),42.8 (4),10.8 (7),36.5 (14),29.6 (5),16.2 (27),14.7 (26),42.3 (7),21.9 (6),15.8 (23),12.3 (30),7.3 (28),7.4 (27),24.9 (1),108.4 (13),-2.6 (12)
6,College Park Skyhawks,36.5 (2),86.3 (6),42.3 (2),9.5 (1),34.6 (7),27.5 (1),14.9 (21),10.5 (5),42.0 (4),21.3 (2),15.0 (27),9.6 (21),5.0 (7),5.7 (14),19.5 (22),100.9 (1),-6.1 (4)
7,Oklahoma City Blue,39.4 (13),90.3 (20),43.7 (8),14.0 (24),45.1 (31),31.0 (7),12.0 (3),13.0 (17),45.6 (20),23.3 (13),16.4 (18),9.7 (22),7.1 (27),4.1 (4),19.7 (19),107.0 (11),-3.0 (10)
8,Stockton Kings,40.7 (19),90.4 (21),45.0 (17),12.4 (13),36.8 (16),33.8 (13),14.2 (17),13.1 (18),43.8 (12),25.2 (24),14.0 (29),8.0 (10),3.6 (1),6.0 (16),20.1 (18),111.8 (21),-4.2 (9)
9,Ciudad de Mexico Capitanes,38.9 (10),90.2 (19),43.1 (5),12.5 (15),42.4 (24),29.5 (4),13.7 (13),15.5 (30),50.9 (29),22.7 (7),18.8 (7),11.6 (29),5.9 (15),5.4 (11),20.2 (17),106.8 (10),2.2 (20)


In [58]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/opponent/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'Opp FGM RANK',
       'Opp FGA RANK', 'Opp FG% RANK', 'Opp 3PM RANK', 'Opp 3PA RANK',
       'Opp 3P% RANK', 'Opp FTM RANK', 'Opp FTA RANK', 'Opp FT% RANK',
       'Opp OREB RANK', 'Opp DREB RANK', 'Opp REB RANK', 'Opp AST RANK',
       'Opp TOV RANK', 'Opp STL RANK', 'Opp BLK RANK', 'Opp BLKA RANK',
       'Opp PF RANK', 'Opp PFD RANK', 'Opp PTS RANK', '+/- RANK', 'Opp FTM', 'Opp FT%', 'Opp DREB', 'Opp PFD'
]

team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Check for NaN or inf values
#print("Checking for NaN and inf values...")
#print(team_stats[numeric_cols].isna().sum())
#print((team_stats[numeric_cols] == np.inf).sum())
#print((team_stats[numeric_cols] == -np.inf).sum())

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['Opp PF', 'Opp TOV']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Raptors 905']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,Opp FGM,Opp FGA,Opp FG%,Opp 3PM,Opp 3PA,Opp 3P%,Opp FTA,Opp OREB,Opp REB,Opp AST,Opp TOV,Opp STL,Opp BLK,Opp BLKA,Opp PF,Opp PTS,+/-
26,Raptors 905,41.4 (22),87.4 (10),47.3 (26),13.3 (20),34.9 (10),38.0 (29),18.4 (31),14.0 (24),47.5 (24),24.6 (20),19.0 (5),7.5 (4),5.5 (10),6.6 (23),21.8 (6),117.6 (29),5.4 (26)


In [59]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/opponent/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'Opp FGM RANK',
       'Opp FGA RANK', 'Opp FG% RANK', 'Opp 3PM RANK', 'Opp 3PA RANK',
       'Opp 3P% RANK', 'Opp FTM RANK', 'Opp FTA RANK', 'Opp FT% RANK',
       'Opp OREB RANK', 'Opp DREB RANK', 'Opp REB RANK', 'Opp AST RANK',
       'Opp TOV RANK', 'Opp STL RANK', 'Opp BLK RANK', 'Opp BLKA RANK',
       'Opp PF RANK', 'Opp PFD RANK', 'Opp PTS RANK', '+/- RANK', 'Opp FTM', 'Opp FT%', 'Opp DREB', 'Opp PFD'
]

team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Check for NaN or inf values
#print("Checking for NaN and inf values...")
#print(team_stats[numeric_cols].isna().sum())
#print((team_stats[numeric_cols] == np.inf).sum())
#print((team_stats[numeric_cols] == -np.inf).sum())

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['Opp PF', 'Opp TOV']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for opponent 905 only
opponent_row = df_with_ranks[df_with_ranks['TEAM'] == 'Westchester Knicks']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the opponent 905 row
styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,Opp FGM,Opp FGA,Opp FG%,Opp 3PM,Opp 3PA,Opp 3P%,Opp FTA,Opp OREB,Opp REB,Opp AST,Opp TOV,Opp STL,Opp BLK,Opp BLKA,Opp PF,Opp PTS,+/-
19,Westchester Knicks,42.0 (25),93.4 (28),44.9 (16),11.7 (10),35.9 (12),32.5 (10),15.3 (22),13.8 (22),44.1 (12),22.7 (8),17.4 (14),8.1 (9),8.4 (31),7.8 (28),21.8 (6),113.8 (24),-7.3 (3)


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/defense/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK',
       'DEF RTG RANK', 'DREB RANK', 'DREB% RANK', 'STL RANK', 'BLK RANK',
       'OPP PTS OFF TOV RANK', 'OPP PTS 2ND CHANCE RANK', 'OPP PTS FB RANK',
       'OPP PTS PAINT RANK' 
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['DEF RTG', 'OPP PTS OFF TOV', 'OPP PTS 2ND CHANCE', 'OPP PTS FB', 'OPP PTS PAINT']  # Columns with descending rank
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,DEF RTG,DREB,DREB%,STL,BLK,OPP PTS OFF TOV,OPP PTS 2ND CHANCE,OPP PTS FB,OPP PTS PAINT
0,Iowa Wolves,101.6 (1),34.9 (5),72.3 (6),10.0 (7),6.7 (7),19.4 (15),11.6 (2),16.7 (25),49.8 (16)
1,Austin Spurs,105.8 (6),30.9 (25),71.7 (8),12.8 (1),5.7 (17),18.4 (8),11.8 (4),11.2 (4),47.6 (11)
2,Capital City Go-Go,105.9 (8),34.7 (6),65.6 (26),7.0 (29),7.2 (5),24.1 (28),16.6 (22),14.4 (16),54.2 (26)
3,Greensboro Swarm,104.9 (5),34.4 (8),72.9 (2),8.6 (18),4.8 (24),17.4 (5),12.4 (5),11.5 (5),44.6 (3)
4,Oklahoma City Blue,106.1 (9),34.4 (8),68.8 (14),9.1 (15),3.9 (30),20.3 (19),16.5 (21),10.6 (3),45.4 (8)
5,Sioux Falls Skyforce,101.6 (1),36.3 (2),74.0 (1),10.5 (5),5.0 (22),16.1 (2),13.6 (8),13.6 (14),52.2 (20)
6,Valley Suns,111.4 (23),33.1 (17),65.7 (25),8.1 (24),4.8 (24),18.6 (9),22.7 (31),14.6 (19),51.5 (19)
7,College Park Skyhawks,104.6 (4),34.4 (8),72.5 (5),7.3 (28),5.5 (19),20.7 (22),10.5 (1),9.7 (2),49.6 (15)
8,Stockton Kings,112.0 (24),33.3 (14),66.9 (21),7.4 (27),6.1 (14),19.8 (17),16.4 (19),17.1 (27),52.4 (22)
9,Ciudad de Mexico Capitanes,104.2 (3),32.3 (20),66.4 (22),9.5 (10),5.2 (21),23.6 (27),15.1 (14),17.6 (29),48.4 (13)


In [60]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/defense/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK',
       'DEF RTG RANK', 'DREB RANK', 'DREB% RANK', 'STL RANK', 'BLK RANK',
       'OPP PTS OFF TOV RANK', 'OPP PTS 2ND CHANCE RANK', 'OPP PTS FB RANK',
       'OPP PTS PAINT RANK' 
]

team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Check for NaN or inf values
#print("Checking for NaN and inf values...")
#print(team_stats[numeric_cols].isna().sum())
#print((team_stats[numeric_cols] == np.inf).sum())
#print((team_stats[numeric_cols] == -np.inf).sum())

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['DEF RTG', 'OPP PTS OFF TOV', 'OPP PTS 2ND CHANCE', 'OPP PTS FB', 'OPP PTS PAINT']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Raptors 905']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,DEF RTG,DREB,DREB%,STL,BLK,OPP PTS OFF TOV,OPP PTS 2ND CHANCE,OPP PTS FB,OPP PTS PAINT
26,Raptors 905,112.9 (28),29.1 (30),64.8 (27),11.3 (4),6.6 (8),21.2 (23),17.5 (26),13.0 (9),52.2 (21)


In [61]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/defense/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK',
       'DEF RTG RANK', 'DREB RANK', 'DREB% RANK', 'STL RANK', 'BLK RANK',
       'OPP PTS OFF TOV RANK', 'OPP PTS 2ND CHANCE RANK', 'OPP PTS FB RANK',
       'OPP PTS PAINT RANK' 
]

team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Check for NaN or inf values
#print("Checking for NaN and inf values...")
#print(team_stats[numeric_cols].isna().sum())
#print((team_stats[numeric_cols] == np.inf).sum())
#print((team_stats[numeric_cols] == -np.inf).sum())

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['DEF RTG', 'OPP PTS OFF TOV', 'OPP PTS 2ND CHANCE', 'OPP PTS FB', 'OPP PTS PAINT']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for opponent 905 only
opponent_row = df_with_ranks[df_with_ranks['TEAM'] == 'Westchester Knicks']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the opponent 905 row
styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,DEF RTG,DREB,DREB%,STL,BLK,OPP PTS OFF TOV,OPP PTS 2ND CHANCE,OPP PTS FB,OPP PTS PAINT
19,Westchester Knicks,107.5 (13),35.0 (4),67.9 (18),9.8 (8),7.8 (4),21.2 (23),16.2 (17),13.8 (14),55.1 (27)


# Clutch

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-traditional/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'WIN% RANK', 'MIN RANK', 'PTS RANK',
    'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
    'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
    'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'BLKA RANK',
    'PF RANK', 'PFD RANK', '+/- RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['PF']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
0,Greensboro Swarm,8.6 (11),2.0 (20),5.0 (16),40.0 (17),0.4 (23),1.6 (22),25.0 (18),2.8 (4),3.6 (4),77.8 (16),1.6 (2),2.8 (8),4.4 (5),0.6 (25),1.6 (4),0.6 (10),0.6 (9),0.4 (12),1.4 (5),3.0 (1),5.2 (1)
1,Oklahoma City Blue,9.0 (8),2.1 (19),5.3 (14),40.5 (15),1.0 (5),2.6 (11),38.9 (6),3.3 (2),3.9 (3),85.2 (8),0.9 (15),2.4 (12),3.3 (12),0.9 (22),1.6 (4),0.9 (2),0.0 (27),0.7 (5),1.6 (9),2.9 (5),2.1 (8)
2,Iowa Wolves,9.8 (4),3.2 (3),7.4 (6),43.2 (14),0.6 (15),2.6 (11),23.1 (19),2.0 (10),2.2 (15),90.9 (6),1.6 (2),4.0 (1),5.6 (1),1.4 (9),1.6 (4),0.8 (4),0.6 (9),0.0 (26),2.4 (25),2.2 (11),2.8 (3)
3,Motor City Cruise,9.2 (6),3.2 (3),7.0 (9),45.7 (9),1.4 (2),3.0 (8),46.7 (3),1.0 (22),1.2 (24),83.3 (11),0.8 (16),2.6 (10),3.4 (10),2.0 (5),1.0 (17),0.8 (4),0.2 (22),0.8 (3),1.4 (5),1.4 (21),2.8 (3)
4,Stockton Kings,7.2 (16),1.4 (27),3.2 (28),43.8 (13),0.6 (15),1.6 (22),37.5 (7),3.4 (1),4.0 (2),85.0 (9),0.6 (23),1.4 (22),2.0 (24),1.0 (17),0.8 (21),0.6 (10),0.0 (27),0.0 (26),0.6 (2),2.8 (6),3.8 (2)
5,Valley Suns,10.4 (3),3.2 (3),6.2 (11),51.6 (3),1.2 (3),2.6 (11),46.2 (4),2.8 (4),3.0 (6),93.3 (4),0.2 (31),2.4 (12),2.6 (17),1.6 (7),1.2 (12),0.2 (25),0.4 (13),0.2 (21),2.2 (20),3.0 (1),1.8 (9)
6,Grand Rapids Gold,8.7 (10),3.0 (6),7.6 (4),39.6 (18),0.7 (12),3.1 (6),22.7 (20),1.6 (16),2.0 (19),78.6 (14),1.6 (2),3.1 (4),4.7 (3),1.4 (9),1.1 (15),0.4 (18),0.7 (7),0.4 (12),2.0 (15),2.1 (12),-0.3 (15)
7,Austin Spurs,9.7 (5),3.0 (6),8.0 (2),37.5 (23),0.7 (12),3.7 (2),18.2 (23),3.0 (3),4.3 (1),69.2 (22),2.0 (1),2.3 (14),4.3 (6),1.0 (17),2.0 (1),0.3 (23),1.0 (1),1.0 (1),1.7 (11),3.0 (1),-0.7 (17)
8,Ciudad de Mexico Capitanes,8.5 (12),2.7 (10),6.8 (10),39.0 (21),1.0 (5),3.0 (8),33.3 (11),2.0 (10),2.2 (15),92.3 (5),1.3 (8),1.8 (20),3.2 (14),1.0 (17),1.3 (11),0.8 (4),0.3 (18),0.8 (3),2.0 (15),2.0 (15),2.2 (7)
9,Rio Grande Valley Vipers,8.3 (13),2.3 (15),4.0 (27),58.3 (1),0.7 (12),1.3 (28),50.0 (2),2.3 (7),3.0 (6),77.8 (16),0.3 (28),3.0 (6),3.3 (12),1.3 (12),0.0 (31),0.0 (28),1.0 (1),0.0 (26),2.0 (15),2.7 (8),2.3 (6)


In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-traditional/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'WIN% RANK', 'MIN RANK', 'PTS RANK',
    'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
    'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
    'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'BLKA RANK',
    'PF RANK', 'PFD RANK', '+/- RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['PF']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Raptors 905']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
26,Raptors 905,6.4 (20),2.4 (13),4.8 (18),50.0 (4),1.0 (5),2.4 (16),41.7 (5),0.6 (26),0.8 (26),75.0 (20),0.4 (26),2.0 (17),2.4 (21),2.2 (3),1.6 (4),0.8 (4),0.4 (13),0.0 (26),2.2 (20),1.0 (25),-3.0 (29)


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-traditional/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'WIN% RANK', 'MIN RANK', 'PTS RANK',
    'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK',
    'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK',
    'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'BLKA RANK',
    'PF RANK', 'PFD RANK', '+/- RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['PF']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for opponent 905 only
opponent_row = df_with_ranks[df_with_ranks['TEAM'] == 'Westchester Knicks']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the opponent 905 row
styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
28,Westchester Knicks,4.6 (27),1.6 (24),3.0 (29),53.3 (2),0.4 (23),0.6 (30),66.7 (1),1.0 (22),1.2 (24),83.3 (11),0.8 (16),1.0 (27),1.8 (28),0.6 (25),1.2 (12),0.4 (18),0.6 (9),0.2 (21),2.2 (20),1.2 (24),-4.2 (31)


In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-advanced/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK',
       'MIN RANK', 'OffRtg RANK', 'DefRtg RANK', 'NetRtg RANK', 'AST% RANK',
       'AST/TO RANK', 'AST Ratio RANK', 'OREB% RANK', 'DREB% RANK',
       'REB% RANK', 'TO Ratio RANK', 'eFG% RANK', 'TS% RANK', 'PACE RANK',
       'PIE RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['DefRtg', 'TOV%']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,OffRtg,DefRtg,NetRtg,AST%,AST/TO,AST Ratio,OREB%,DREB%,REB%,TOV%,eFG%,TS%,PACE,PIE
0,Oklahoma City Blue,110.5 (11),90.6 (7),20.0 (8),40.0 (23),0.55 (25),8.5 (26),33.3 (19),65.5 (15),50.9 (14),19.3 (24),50.0 (11),64.4 (7),108.57 (6),70.7 (6)
1,Capital City Go-Go,127.5 (3),125.5 (26),2.0 (12),52.6 (12),2.0 (3),14.1 (14),40.0 (11),54.2 (24),46.9 (23),9.8 (5),54.9 (7),65.8 (4),109.16 (3),51.9 (12)
2,Delaware Blue Coats,100.0 (17),79.5 (4),20.5 (7),60.0 (9),1.0 (16),12.7 (16),30.8 (21),59.4 (22),46.6 (25),16.7 (16),42.1 (24),57.2 (12),105.9 (9),81.9 (4)
3,Grand Rapids Gold,103.4 (14),108.6 (18),-5.2 (15),47.6 (20),1.25 (11),12.3 (17),35.3 (16),66.7 (12),51.4 (13),13.6 (10),44.3 (20),51.6 (19),91.32 (28),46.8 (14)
4,Greensboro Swarm,116.2 (7),53.1 (1),63.1 (1),30.0 (31),0.38 (30),5.9 (31),61.1 (2),84.2 (3),73.0 (2),21.6 (27),44.0 (21),65.3 (5),92.39 (27),160.5 (1)
5,Ciudad de Mexico Capitanes,110.9 (10),84.4 (5),26.4 (6),37.5 (26),0.75 (22),10.0 (23),38.5 (14),60.9 (21),49.0 (19),17.4 (20),46.3 (17),54.6 (15),98.97 (20),62.7 (9)
6,College Park Skyhawks,75.5 (30),93.6 (8),-18.1 (21),40.0 (23),0.4 (29),7.1 (29),12.0 (31),57.1 (23),35.8 (30),20.4 (25),36.4 (28),47.2 (27),105.71 (10),27.1 (27)
7,Iowa Wolves,108.9 (12),77.8 (2),31.1 (5),43.8 (21),0.88 (20),11.5 (18),40.9 (10),87.5 (2),65.2 (3),17.8 (21),47.3 (15),58.6 (11),101.12 (14),75.7 (5)
8,Motor City Cruise,115.0 (8),78.0 (3),37.0 (3),62.5 (7),2.0 (3),18.2 (2),25.0 (25),65.0 (17),45.0 (26),12.5 (8),55.7 (5),61.1 (9),94.22 (23),70.7 (6)
9,Stockton Kings,128.6 (2),94.4 (9),34.1 (4),71.4 (3),1.25 (11),13.5 (15),44.4 (6),63.6 (20),55.0 (7),14.3 (12),53.1 (8),72.6 (2),100.94 (15),104.7 (2)


In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/advanced/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK',
       'MIN RANK', 'OffRtg RANK', 'DefRtg RANK', 'NetRtg RANK', 'AST% RANK',
       'AST/TO RANK', 'AST Ratio RANK', 'OREB% RANK', 'DREB% RANK',
       'REB% RANK', 'TO Ratio RANK', 'eFG% RANK', 'TS% RANK', 'PACE RANK',
       'PIE RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['DefRtg', 'TOV%']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Raptors 905']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,OffRtg,DefRtg,NetRtg,AST%,AST/TO,AST Ratio,OREB%,DREB%,REB%,TOV%,eFG%,TS%,PACE,PIE
24,Raptors 905,108.4 (18),112.9 (28),-4.5 (23),63.0 (6),1.55 (7),18.6 (3),29.6 (24),64.8 (27),46.7 (27),16.9 (18),54.8 (6),58.5 (12),103.86 (8),47.9 (24)


In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/advanced/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK',
       'MIN RANK', 'OffRtg RANK', 'DefRtg RANK', 'NetRtg RANK', 'AST% RANK',
       'AST/TO RANK', 'AST Ratio RANK', 'OREB% RANK', 'DREB% RANK',
       'REB% RANK', 'TO Ratio RANK', 'eFG% RANK', 'TS% RANK', 'PACE RANK',
       'PIE RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['DefRtg', 'TOV%']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for opponent 905 only
opponent_row = df_with_ranks[df_with_ranks['TEAM'] == 'Westchester Knicks']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the opponent 905 row
styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,OffRtg,DefRtg,NetRtg,AST%,AST/TO,AST Ratio,OREB%,DREB%,REB%,TOV%,eFG%,TS%,PACE,PIE
16,Westchester Knicks,114.3 (3),107.5 (13),6.8 (4),56.7 (23),1.54 (12),17.2 (14),39.4 (1),67.9 (17),53.6 (3),16.2 (12),53.6 (8),58.1 (15),105.94 (3),54.8 (3)


In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-four-factors/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'eFG% RANK', 'TO Ratio RANK',
    'OREB% RANK', 'Opp FTA Rate RANK', 'Opp To Ratio RANK', 'WIN% RANK', 'FTA Rate RANK', 'Opp eFG% RANK', 'Opp OREB% RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['TOV%', 'Opp eFG%', 'Opp FTA Rate', 'Opp OREB%']  # Columns with descending rank
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,eFG%,FTA Rate,TOV%,OREB%,Opp eFG%,Opp FTA Rate,Opp TOV%,Opp OREB%
0,Greensboro Swarm,44.0 (21),0.72 (4),21.6 (27),61.1 (2),25.0 (1),0.25 (9),25.0 (4),15.8 (3)
1,Oklahoma City Blue,50.0 (11),0.73 (3),19.3 (24),33.3 (19),40.5 (8),0.31 (14),15.1 (16),34.5 (15)
2,Iowa Wolves,47.3 (15),0.297 (22),17.8 (21),40.9 (10),36.4 (3),0.273 (10),17.8 (9),12.5 (2)
3,Motor City Cruise,55.7 (5),0.171 (27),12.5 (8),25.0 (25),40.6 (9),0.219 (4),24.4 (5),35.0 (17)
4,Stockton Kings,53.1 (8),1.25 (1),14.3 (12),44.4 (6),41.2 (10),0.235 (7),16.7 (11),36.4 (20)
5,Valley Suns,61.3 (2),0.484 (9),14.0 (11),20.0 (29),48.6 (18),0.389 (16),14.3 (18),34.8 (16)
6,Grand Rapids Gold,44.3 (20),0.264 (25),13.6 (10),35.3 (16),49.1 (19),0.283 (12),8.6 (25),33.3 (12)
7,Austin Spurs,41.7 (25),0.542 (8),21.4 (26),56.3 (3),46.3 (16),0.222 (5),12.5 (22),56.3 (29)
8,Ciudad de Mexico Capitanes,46.3 (17),0.317 (18),17.4 (20),38.5 (14),43.9 (12),0.424 (18),26.7 (2),39.1 (21)
9,Rio Grande Valley Vipers,66.7 (1),0.75 (2),0.0 (1),28.6 (23),38.2 (6),0.294 (13),14.3 (18),23.1 (5)


In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-four-factors/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'eFG% RANK', 'TO Ratio RANK',
    'OREB% RANK', 'Opp FTA Rate RANK', 'Opp To Ratio RANK', 'WIN% RANK', 'FTA Rate RANK', 'Opp eFG% RANK', 'Opp OREB% RANK'
]

team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Check for NaN or inf values
#print("Checking for NaN and inf values...")
#print(team_stats[numeric_cols].isna().sum())
#print((team_stats[numeric_cols] == np.inf).sum())
#print((team_stats[numeric_cols] == -np.inf).sum())

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['TOV%', 'Opp eFG%', 'Opp FTA Rate', 'Opp OREB%']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Raptors 905']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,eFG%,FTA Rate,TOV%,OREB%,Opp eFG%,Opp FTA Rate,Opp TOV%,Opp OREB%
26,Raptors 905,60.4 (3),0.167 (28),25.8 (28),25.0 (25),55.8 (24),0.5 (22),13.2 (21),26.7 (7)


In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-four-factors/?sort=W_PCT&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'WIN%', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'eFG% RANK', 'TO Ratio RANK',
    'OREB% RANK', 'Opp FTA Rate RANK', 'Opp To Ratio RANK', 'WIN% RANK', 'FTA Rate RANK', 'Opp eFG% RANK', 'Opp OREB% RANK'
]

team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Check for NaN or inf values
#print("Checking for NaN and inf values...")
#print(team_stats[numeric_cols].isna().sum())
#print((team_stats[numeric_cols] == np.inf).sum())
#print((team_stats[numeric_cols] == -np.inf).sum())

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['TOV%', 'Opp eFG%', 'Opp FTA Rate', 'Opp OREB%']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for opponent 905 only
opponent_row = df_with_ranks[df_with_ranks['TEAM'] == 'Westchester Knicks']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the opponent 905 row
styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,eFG%,FTA Rate,TOV%,OREB%,Opp eFG%,Opp FTA Rate,Opp TOV%,Opp OREB%
28,Westchester Knicks,60.0 (4),0.4 (15),30.0 (30),50.0 (4),75.0 (31),0.75 (29),6.9 (29),33.3 (12)


In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-misc/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'PTS OFF TO RANK',
       '2nd PTS RANK', 'FBPs RANK', 'PITP RANK', 'Opp PTS OFF TO RANK',
       'Opp 2nd PTS RANK', 'Opp FBPs RANK', 'Opp PITP RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['Opp PTS OFF TO', 'Opp 2nd PTS', 'Opp FBPs', 'Opp PITP']  # Columns with descending rank
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS OFF TO,2nd PTS,FBPs,PITP,Opp PTS OFF TO,Opp 2nd PTS,Opp FBPs,Opp PITP
0,Oklahoma City Blue,1.4 (13),1.6 (9),0.3 (21),2.3 (23),1.4 (21),2.4 (28),0.0 (1),4.0 (24)
1,Capital City Go-Go,1.9 (6),0.9 (19),0.8 (9),2.8 (15),1.3 (17),2.4 (28),0.5 (12),4.0 (24)
2,Delaware Blue Coats,1.0 (16),0.9 (19),0.8 (9),3.0 (12),1.3 (17),1.1 (17),0.8 (20),2.0 (7)
3,Grand Rapids Gold,1.3 (14),1.3 (11),0.3 (21),3.7 (6),1.4 (21),1.7 (20),0.9 (22),3.4 (20)
4,Greensboro Swarm,2.4 (2),2.0 (5),0.8 (9),2.8 (15),1.2 (13),0.6 (7),0.0 (1),1.2 (1)
5,Ciudad de Mexico Capitanes,1.5 (12),0.7 (26),0.8 (9),2.7 (19),1.3 (17),1.3 (18),0.7 (17),2.0 (7)
6,College Park Skyhawks,0.7 (18),0.4 (29),1.0 (6),1.7 (28),1.4 (21),0.9 (12),0.9 (22),2.3 (11)
7,Iowa Wolves,2.4 (2),2.8 (1),1.6 (2),4.8 (1),0.4 (4),0.4 (3),0.6 (15),3.2 (17)
8,Motor City Cruise,2.4 (2),1.2 (12),0.8 (9),3.6 (7),1.2 (13),0.0 (1),0.4 (7),4.0 (24)
9,Stockton Kings,0.6 (22),1.2 (12),0.0 (24),0.8 (31),0.8 (7),0.6 (7),0.8 (20),2.4 (15)


In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-misc/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'PTS OFF TO RANK',
       '2nd PTS RANK', 'FBPs RANK', 'PITP RANK', 'Opp PTS OFF TO RANK',
       'Opp 2nd PTS RANK', 'Opp FBPs RANK', 'Opp PITP RANK'
]

team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Check for NaN or inf values
#print("Checking for NaN and inf values...")
#print(team_stats[numeric_cols].isna().sum())
#print((team_stats[numeric_cols] == np.inf).sum())
#print((team_stats[numeric_cols] == -np.inf).sum())

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['Opp PTS OFF TO', 'Opp 2nd PTS', 'Opp FBPs', 'Opp PITP']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Raptors 905']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS OFF TO,2nd PTS,FBPs,PITP,Opp PTS OFF TO,Opp 2nd PTS,Opp FBPs,Opp PITP
25,Raptors 905,0.4 (23),0.8 (21),0.4 (18),2.8 (15),1.8 (25),2.0 (26),0.6 (15),1.6 (5)


In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-misc/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'PTS OFF TO RANK',
       '2nd PTS RANK', 'FBPs RANK', 'PITP RANK', 'Opp PTS OFF TO RANK',
       'Opp 2nd PTS RANK', 'Opp FBPs RANK', 'Opp PITP RANK'
]

team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Check for NaN or inf values
#print("Checking for NaN and inf values...")
#print(team_stats[numeric_cols].isna().sum())
#print((team_stats[numeric_cols] == np.inf).sum())
#print((team_stats[numeric_cols] == -np.inf).sum())

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['Opp PTS OFF TO', 'Opp 2nd PTS', 'Opp FBPs', 'Opp PITP']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for opponent 905 only
opponent_row = df_with_ranks[df_with_ranks['TEAM'] == 'Westchester Knicks']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for PF (descending order)
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5            
        else:
            # Normal normalization for ascending order
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the opponent 905 row
styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,PTS OFF TO,2nd PTS,FBPs,PITP,Opp PTS OFF TO,Opp 2nd PTS,Opp FBPs,Opp PITP
28,Westchester Knicks,0.4 (23),0.8 (21),0.0 (24),2.0 (25),2.4 (30),0.4 (3),0.4 (7),3.2 (17)


In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-scoring/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK',
    'MIN RANK', '%FGA 2PT RANK', '%FGA 3PT RANK', '%PTS 2PT RANK',
    '%PTS 2PT- MR RANK', '%PTS 3PT RANK', '%PTS FBPs RANK', '%PTS FT RANK',
    '%PTS OffTO RANK', '%PTS PITP RANK', '2FGM %AST RANK',
    '2FGM %UAST RANK', '3FGM %AST RANK', '3FGM %UAST RANK', 'FGM %AST RANK',
    'FGM %UAST RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (all in descending order since no inversion is needed)
    ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        # Normal normalization for all columns (descending rank)
        norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,%FGA 2PT,%FGA 3PT,%PTS 2PT,%PTS 2PT- MR,%PTS 3PT,%PTS FBPs,%PTS FT,%PTS OffTO,%PTS PITP,2FGM %AST,2FGM %UAST,3FGM %AST,3FGM %UAST,FGM %AST,FGM %UAST
0,Oklahoma City Blue,51.4 (23),48.6 (9),25.4 (30),0.0 (17),33.3 (7),3.2 (23),36.5 (3),15.9 (14),25.4 (30),0.0 (31),100.0 (1),85.7 (15),14.3 (12),40.0 (23),60.0 (7)
1,Capital City Go-Go,53.7 (21),46.3 (11),36.9 (26),3.1 (15),32.3 (11),9.2 (15),24.6 (13),23.1 (10),33.8 (23),33.3 (20),66.7 (11),85.7 (15),14.3 (12),52.6 (12),47.4 (20)
2,Delaware Blue Coats,68.4 (4),31.6 (28),48.1 (14),3.7 (14),11.1 (27),11.1 (9),31.5 (6),14.8 (15),44.4 (10),53.8 (7),46.2 (25),100.0 (1),0.0 (16),60.0 (9),40.0 (22)
3,Grand Rapids Gold,58.5 (11),41.5 (21),52.5 (10),9.8 (6),24.6 (19),3.3 (22),18.0 (22),14.8 (15),42.6 (15),31.3 (22),68.8 (10),100.0 (1),0.0 (16),47.6 (20),52.4 (12)
4,Greensboro Swarm,68.0 (6),32.0 (26),37.2 (24),4.7 (12),14.0 (25),9.3 (14),32.6 (5),27.9 (4),32.6 (25),25.0 (24),75.0 (8),50.0 (24),50.0 (4),30.0 (31),70.0 (1)
5,Ciudad de Mexico Capitanes,56.1 (17),43.9 (14),39.2 (21),7.8 (9),35.3 (5),9.8 (13),23.5 (15),17.6 (13),31.4 (27),10.0 (30),90.0 (2),83.3 (18),16.7 (8),37.5 (26),62.5 (5)
6,College Park Skyhawks,45.5 (30),54.5 (2),32.4 (29),0.0 (17),32.4 (10),18.9 (3),29.7 (8),13.5 (18),32.4 (26),50.0 (8),50.0 (19),25.0 (27),75.0 (2),40.0 (23),60.0 (7)
7,Iowa Wolves,64.9 (9),35.1 (23),53.1 (8),4.1 (13),18.4 (24),16.3 (6),20.4 (20),24.5 (9),49.0 (9),46.2 (14),53.8 (18),33.3 (26),66.7 (3),43.8 (21),56.3 (11)
8,Motor City Cruise,57.1 (15),42.9 (17),39.1 (22),0.0 (17),45.7 (2),8.7 (16),10.9 (27),26.1 (7),39.1 (18),44.4 (16),55.6 (15),85.7 (15),14.3 (12),62.5 (7),37.5 (25)
9,Stockton Kings,50.0 (25),50.0 (6),22.2 (31),11.1 (5),25.0 (17),0.0 (24),47.2 (2),8.3 (22),11.1 (31),50.0 (8),50.0 (19),100.0 (1),0.0 (16),71.4 (3),28.6 (29)


In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-scoring/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK',
    'MIN RANK', '%FGA 2PT RANK', '%FGA 3PT RANK', '%PTS 2PT RANK',
    '%PTS 2PT- MR RANK', '%PTS 3PT RANK', '%PTS FBPs RANK', '%PTS FT RANK',
    '%PTS OffTO RANK', '%PTS PITP RANK', '2FGM %AST RANK',
    '2FGM %UAST RANK', '3FGM %AST RANK', '3FGM %UAST RANK', 'FGM %AST RANK',
    'FGM %UAST RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (all in descending order)
    ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Raptors 905']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        # Normal normalization for all columns
        norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,%FGA 2PT,%FGA 3PT,%PTS 2PT,%PTS 2PT- MR,%PTS 3PT,%PTS FBPs,%PTS FT,%PTS OffTO,%PTS PITP,2FGM %AST,2FGM %UAST,3FGM %AST,3FGM %UAST,FGM %AST,FGM %UAST
25,Raptors 905,50.0 (25),50.0 (6),43.8 (15),0.0 (17),46.9 (1),6.3 (17),9.4 (28),6.3 (26),43.8 (11),85.7 (1),14.3 (31),100.0 (1),0.0 (16),91.7 (1),8.3 (31)


In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-scoring/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Standardize column names
team_stats.columns = team_stats.columns.str.replace('\xa0', ' ', regex=False).str.strip()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK',
    'MIN RANK', '%FGA 2PT RANK', '%FGA 3PT RANK', '%PTS 2PT RANK',
    '%PTS 2PT- MR RANK', '%PTS 3PT RANK', '%PTS FBPs RANK', '%PTS FT RANK',
    '%PTS OffTO RANK', '%PTS PITP RANK', '2FGM %AST RANK',
    '2FGM %UAST RANK', '3FGM %AST RANK', '3FGM %UAST RANK', 'FGM %AST RANK',
    'FGM %UAST RANK'
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (all in descending order)
    ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for opponent 905 only
opponent_row = df_with_ranks[df_with_ranks['TEAM'] == 'Westchester Knicks']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        # Normal normalization for all columns
        norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the opponent 905 row
styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,%FGA 2PT,%FGA 3PT,%PTS 2PT,%PTS 2PT- MR,%PTS 3PT,%PTS FBPs,%PTS FT,%PTS OffTO,%PTS PITP,2FGM %AST,2FGM %UAST,3FGM %AST,3FGM %UAST,FGM %AST,FGM %UAST
28,Westchester Knicks,80.0 (1),20.0 (31),52.2 (11),8.7 (7),26.1 (16),0.0 (24),21.7 (16),8.7 (21),43.5 (13),33.3 (20),66.7 (11),50.0 (24),50.0 (4),37.5 (26),62.5 (5)


In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-opponent/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'Opp FGM RANK',
       'Opp FGA RANK', 'Opp FG% RANK', 'Opp 3PM RANK', 'Opp 3PA RANK',
       'Opp 3P% RANK', 'Opp FTM RANK', 'Opp FTA RANK', 'Opp FT% RANK',
       'Opp OREB RANK', 'Opp DREB RANK', 'Opp REB RANK', 'Opp AST RANK',
       'Opp TOV RANK', 'Opp STL RANK', 'Opp BLK RANK', 'Opp BLKA RANK',
       'Opp PF RANK', 'Opp PFD RANK', 'Opp PTS RANK', '+/- RANK', 'Opp FTM', 'Opp FT%', 'Opp DREB', 'Opp PFD' 
]
team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['Opp PF', 'Opp TOV']  # Columns with descending rank
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,Opp FGM,Opp FGA,Opp FG%,Opp 3PM,Opp 3PA,Opp 3P%,Opp FTA,Opp OREB,Opp REB,Opp AST,Opp TOV,Opp STL,Opp BLK,Opp BLKA,Opp PF,Opp PTS,+/-
0,Oklahoma City Blue,2.3 (17),6.0 (19),38.1 (10),0.3 (4),2.7 (22),10.5 (4),1.9 (11),0.9 (16),2.9 (13),0.6 (5),1.1 (13),0.6 (14),0.7 (26),0.0 (1),2.9 (5),6.9 (15),-2.1 (8)
1,Capital City Go-Go,2.6 (20),5.3 (14),50.0 (25),0.6 (14),1.9 (10),33.3 (18),2.6 (21),1.3 (23),3.0 (15),1.1 (15),0.9 (19),0.3 (5),0.3 (12),0.4 (15),1.8 (20),8.0 (20),-0.1 (13)
2,Delaware Blue Coats,1.4 (5),5.3 (14),26.2 (2),0.4 (7),2.1 (13),17.6 (8),1.3 (7),1.0 (19),2.9 (13),0.9 (10),0.9 (19),0.8 (24),0.4 (16),0.8 (26),2.4 (10),4.4 (4),-2.4 (5)
3,Grand Rapids Gold,3.1 (26),7.6 (28),41.5 (17),1.1 (29),3.4 (29),33.3 (18),2.1 (14),1.4 (26),4.3 (26),2.0 (29),0.7 (24),0.9 (29),0.4 (16),0.7 (24),2.1 (12),9.0 (23),0.3 (15)
4,Greensboro Swarm,1.0 (1),4.8 (10),20.8 (1),0.4 (7),2.6 (18),15.4 (6),1.2 (4),0.4 (4),1.6 (5),0.2 (1),1.6 (5),0.8 (24),0.4 (16),0.6 (21),3.0 (1),3.4 (2),-5.2 (1)
5,Ciudad de Mexico Capitanes,2.0 (12),5.5 (17),36.4 (8),0.8 (17),3.0 (23),27.8 (14),2.3 (16),0.8 (13),3.5 (20),1.0 (12),2.0 (1),0.8 (24),0.8 (28),0.3 (11),2.0 (15),6.3 (11),-2.2 (7)
6,College Park Skyhawks,1.7 (9),5.1 (11),33.3 (6),0.4 (7),2.4 (14),17.6 (8),2.4 (19),1.3 (23),3.9 (25),0.9 (10),1.3 (9),0.7 (20),0.3 (12),0.1 (6),2.1 (12),6.3 (11),1.0 (20)
7,Iowa Wolves,2.2 (14),6.6 (22),33.3 (6),0.4 (7),3.0 (23),13.3 (5),1.8 (10),0.4 (4),2.8 (12),1.4 (17),1.6 (5),0.4 (8),0.0 (1),0.6 (21),2.2 (11),7.0 (16),-2.8 (3)
8,Motor City Cruise,2.6 (20),6.4 (21),40.6 (14),0.0 (1),2.0 (11),0.0 (1),1.4 (8),0.8 (13),3.6 (21),0.8 (7),2.0 (1),0.4 (8),0.8 (28),0.2 (7),1.4 (21),6.4 (13),-2.8 (3)
9,Stockton Kings,1.4 (5),3.4 (4),41.2 (16),0.0 (1),2.0 (11),0.0 (1),0.8 (2),0.6 (8),1.2 (2),0.4 (3),0.6 (25),0.6 (14),0.0 (1),0.0 (1),2.8 (6),3.4 (2),-3.8 (2)


In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-opponent/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'Opp FGM RANK',
       'Opp FGA RANK', 'Opp FG% RANK', 'Opp 3PM RANK', 'Opp 3PA RANK',
       'Opp 3P% RANK', 'Opp FTM RANK', 'Opp FTA RANK', 'Opp FT% RANK',
       'Opp OREB RANK', 'Opp DREB RANK', 'Opp REB RANK', 'Opp AST RANK',
       'Opp TOV RANK', 'Opp STL RANK', 'Opp BLK RANK', 'Opp BLKA RANK',
       'Opp PF RANK', 'Opp PFD RANK', 'Opp PTS RANK', '+/- RANK', 'Opp FTM', 'Opp FT%', 'Opp DREB', 'Opp PFD'
]

team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Check for NaN or inf values
#print("Checking for NaN and inf values...")
#print(team_stats[numeric_cols].isna().sum())
#print((team_stats[numeric_cols] == np.inf).sum())
#print((team_stats[numeric_cols] == -np.inf).sum())

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['Opp PF', 'Opp TOV']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for Raptors 905 only
raptors_row = df_with_ranks[df_with_ranks['TEAM'] == 'Raptors 905']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the Raptors 905 row
styled_df = raptors_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,Opp FGM,Opp FGA,Opp FG%,Opp 3PM,Opp 3PA,Opp 3P%,Opp FTA,Opp OREB,Opp REB,Opp AST,Opp TOV,Opp STL,Opp BLK,Opp BLKA,Opp PF,Opp PTS,+/-
25,Raptors 905,2.2 (14),5.2 (12),42.3 (19),1.4 (31),2.4 (14),58.3 (29),2.6 (21),0.6 (8),2.4 (10),1.6 (22),1.0 (15),1.0 (30),0.0 (1),0.4 (15),1.0 (25),9.4 (25),3.0 (29)


In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/clutch-opponent/?sort=W&dir=-1"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# Drop the specified columns
columns_to_drop = [
    'Unnamed: 0', 'GP', 'W', 'L', 'MIN', 'GP RANK', 'W RANK', 'L RANK', 'MIN RANK', 'Opp FGM RANK',
       'Opp FGA RANK', 'Opp FG% RANK', 'Opp 3PM RANK', 'Opp 3PA RANK',
       'Opp 3P% RANK', 'Opp FTM RANK', 'Opp FTA RANK', 'Opp FT% RANK',
       'Opp OREB RANK', 'Opp DREB RANK', 'Opp REB RANK', 'Opp AST RANK',
       'Opp TOV RANK', 'Opp STL RANK', 'Opp BLK RANK', 'Opp BLKA RANK',
       'Opp PF RANK', 'Opp PFD RANK', 'Opp PTS RANK', '+/- RANK', 'Opp FTM', 'Opp FT%', 'Opp DREB', 'Opp PFD'
]

team_stats = team_stats.drop(columns=[col for col in columns_to_drop if col in team_stats.columns])

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns

# Check for NaN or inf values
#print("Checking for NaN and inf values...")
#print(team_stats[numeric_cols].isna().sum())
#print((team_stats[numeric_cols] == np.inf).sum())
#print((team_stats[numeric_cols] == -np.inf).sum())

# Calculate ranks and append ranks to numeric columns for display
invert_cols = ['Opp PF', 'Opp TOV']  # PF will be descending, all others ascending
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (invert order for specified columns)
    if col in invert_cols:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    else:
        ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Filter for opponent 905 only
opponent_row = df_with_ranks[df_with_ranks['TEAM'] == 'Westchester Knicks']

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        if col in invert_cols:
            # Invert normalization for specified columns
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        else:
            # Normal normalization
            norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Style the opponent 905 row
styled_df = opponent_row.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

,TEAM,Opp FGM,Opp FGA,Opp FG%,Opp 3PM,Opp 3PA,Opp 3P%,Opp FTA,Opp OREB,Opp REB,Opp AST,Opp TOV,Opp STL,Opp BLK,Opp BLKA,Opp PF,Opp PTS,+/-
28,Westchester Knicks,2.6 (20),4.0 (6),65.0 (31),0.8 (17),1.2 (4),66.7 (31),3.0 (27),0.4 (4),1.2 (2),1.4 (17),0.4 (28),0.6 (14),0.2 (7),0.6 (21),1.2 (24),8.8 (22),4.2 (31)


# Shooting

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/shooting/"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# --- New Code Integration: Drop Specific MultiIndex Levels ---
levels_to_drop = [1, 2, 4, 5]  # Specify the level indices to drop (1-based indexing)

# Check if the columns are MultiIndex
if isinstance(team_stats.columns, pd.MultiIndex):
    # Drop the specified levels
    team_stats.columns = team_stats.columns.droplevel(levels_to_drop)
    #print("Dropped levels:", levels_to_drop)
    
    # Replace 'Unnamed: 0_level_0' with 'TEAM' in MultiIndex
    new_columns = []
    for col in team_stats.columns:
        if col[0] == 'Unnamed: 0_level_0':
            new_columns.append((' ', col[1]))
        else:
            new_columns.append(col)
    team_stats.columns = pd.MultiIndex.from_tuples(new_columns)
    #print("Updated columns after replacing 'Unnamed: 0_level_0' with 'TEAM':")
else:
    print("Columns are not MultiIndex. No levels to drop.")

# Drop rows with NaN values
team_stats = team_stats.dropna()

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns.tolist()

# Handle FG% columns: convert to numeric if they contain strings
for col in team_stats.columns:
    if 'FG%' in col and col not in numeric_cols:
        team_stats[col] = pd.to_numeric(team_stats[col], errors='coerce')
        numeric_cols.append(col)

# Re-check numeric columns after converting FG% columns
numeric_cols = team_stats.select_dtypes(include=['number']).columns.tolist()

# Calculate ranks and append ranks to numeric columns for display
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (all in descending order since no inversion is needed)
    ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        # Normal normalization for all columns (descending rank)
        norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/shooting/"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# --- New Code Integration: Drop Specific MultiIndex Levels ---
levels_to_drop = [1, 2, 4, 5]  # Specify the level indices to drop (1-based indexing)

# Check if the columns are MultiIndex
if isinstance(team_stats.columns, pd.MultiIndex):
    # Drop the specified levels
    team_stats.columns = team_stats.columns.droplevel(levels_to_drop)
    #print("Dropped levels:", levels_to_drop)
    
    # Replace 'Unnamed: 0_level_0' with 'TEAM' in MultiIndex
    new_columns = []
    for col in team_stats.columns:
        if col[0] == 'Unnamed: 0_level_0':
            new_columns.append((' ', col[1]))
        else:
            new_columns.append(col)
    team_stats.columns = pd.MultiIndex.from_tuples(new_columns)
    #print("Updated columns after replacing 'Unnamed: 0_level_0' with 'TEAM':")
else:
    print("Columns are not MultiIndex. No levels to drop.")

# Drop rows with NaN values
team_stats = team_stats.dropna()

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns.tolist()

# Handle FG% columns: convert to numeric if they contain strings
for col in team_stats.columns:
    if 'FG%' in col and col not in numeric_cols:
        team_stats[col] = pd.to_numeric(team_stats[col], errors='coerce')
        numeric_cols.append(col)

# Re-check numeric columns after converting FG% columns
numeric_cols = team_stats.select_dtypes(include=['number']).columns.tolist()

# Calculate ranks and append ranks to numeric columns for display
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (all in descending order since no inversion is needed)
    ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        # Normal normalization for all columns (descending rank)
        norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Filter for 'Raptors 905' team
raptors_905_df = df_with_ranks[df_with_ranks[(' ', 'TEAM')] == 'Raptors 905']

# Apply the styling only to 'Raptors 905' row
styled_raptors_df = raptors_905_df.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame for 'Raptors 905'
styled_raptors_df

In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/shooting/"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# --- New Code Integration: Drop Specific MultiIndex Levels ---
levels_to_drop = [1, 2, 4, 5]  # Specify the level indices to drop (1-based indexing)

# Check if the columns are MultiIndex
if isinstance(team_stats.columns, pd.MultiIndex):
    # Drop the specified levels
    team_stats.columns = team_stats.columns.droplevel(levels_to_drop)
    #print("Dropped levels:", levels_to_drop)
    
    # Replace 'Unnamed: 0_level_0' with 'TEAM' in MultiIndex
    new_columns = []
    for col in team_stats.columns:
        if col[0] == 'Unnamed: 0_level_0':
            new_columns.append((' ', col[1]))
        else:
            new_columns.append(col)
    team_stats.columns = pd.MultiIndex.from_tuples(new_columns)
    #print("Updated columns after replacing 'Unnamed: 0_level_0' with 'TEAM':")
else:
    print("Columns are not MultiIndex. No levels to drop.")

# Drop rows with NaN values
team_stats = team_stats.dropna()

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns.tolist()

# Handle FG% columns: convert to numeric if they contain strings
for col in team_stats.columns:
    if 'FG%' in col and col not in numeric_cols:
        team_stats[col] = pd.to_numeric(team_stats[col], errors='coerce')
        numeric_cols.append(col)

# Re-check numeric columns after converting FG% columns
numeric_cols = team_stats.select_dtypes(include=['number']).columns.tolist()

# Calculate ranks and append ranks to numeric columns for display
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (all in descending order since no inversion is needed)
    ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        # Normal normalization for all columns (descending rank)
        norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Filter for 'Raptors 905' team
raptors_905_df = df_with_ranks[df_with_ranks[(' ', 'TEAM')] == 'Westchester Knicks']

# Apply the styling only to 'Raptors 905' row
styled_raptors_df = raptors_905_df.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame for 'Raptors 905'
styled_raptors_df

In [21]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashteamshotlocations"

# Define the headers from the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters for the 8ft range
params = {
    "Conference": "",
    "DateFrom": "",
    "DateTo": "",
    "DistanceRange": "8ft Range",
    "Division": "",
    "GameScope": "",
    "GameSegment": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

# Verify if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    result_set = data["resultSets"]["rowSet"]
    columns = data["resultSets"]["headers"][1]["columnNames"]

    # Convert to a DataFrame
    team_stats = pd.DataFrame(result_set, columns=columns)

    # Drop the TEAM_ID column
    if "TEAM_ID" in team_stats.columns:
        team_stats = team_stats.drop(columns=["TEAM_ID"])

    # Rename FG_PCT columns to FG% and TEAM_NAME column to TEAM
    team_stats = team_stats.rename(columns=lambda x: x.replace("FG_PCT", "FG%") if "FG_PCT" in x else x)
    team_stats = team_stats.rename(columns={"TEAM_NAME": "TEAM"})

    # Add top-level column names to create a MultiIndex
    top_level = [" "] + ["Less Than 8ft.", "Less Than 8ft.", "Less Than 8ft.",
                         "8-16 ft.", "8-16 ft.", "8-16 ft.",
                         "16-24 ft.", "16-24 ft.", "16-24 ft.",
                         "24+ ft.", "24+ ft.", "24+ ft.",
                         "Back Court Shot", "Back Court Shot", "Back Court Shot"]

    # Combine the new top level with the current column names
    multi_index = pd.MultiIndex.from_tuples(zip(top_level, team_stats.columns))

    # Apply the MultiIndex to the DataFrame
    team_stats.columns = multi_index

    # Drop the FGM, FGA, and FG% columns under the "Back Court Shot" header
    team_stats = team_stats.drop(columns=["Back Court Shot"], level=0)

    # Apply conditional formatting and ranking
    numeric_cols = [col for col in team_stats.columns if col[1] in ["FGM", "FGA", "FG%"]]
    df_with_ranks = team_stats.copy()

    # Add ranks
    for col in numeric_cols:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

    # Apply color formatting
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Style the DataFrame
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

In [21]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashteamshotlocations"

# Define the headers from the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters for the 8ft range
params = {
    "Conference": "",
    "DateFrom": "",
    "DateTo": "",
    "DistanceRange": "8ft Range",
    "Division": "",
    "GameScope": "",
    "GameSegment": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

# Verify if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    result_set = data["resultSets"]["rowSet"]
    columns = data["resultSets"]["headers"][1]["columnNames"]

    # Convert to a DataFrame
    team_stats = pd.DataFrame(result_set, columns=columns)

    # Drop the TEAM_ID column
    if "TEAM_ID" in team_stats.columns:
        team_stats = team_stats.drop(columns=["TEAM_ID"])

    # Rename FG_PCT columns to FG% and TEAM_NAME column to TEAM
    team_stats = team_stats.rename(columns=lambda x: x.replace("FG_PCT", "FG%") if "FG_PCT" in x else x)
    team_stats = team_stats.rename(columns={"TEAM_NAME": "TEAM"})

    # Add top-level column names to create a MultiIndex
    top_level = [" "] + ["Less Than 8ft.", "Less Than 8ft.", "Less Than 8ft.",
                         "8-16 ft.", "8-16 ft.", "8-16 ft.",
                         "16-24 ft.", "16-24 ft.", "16-24 ft.",
                         "24+ ft.", "24+ ft.", "24+ ft.",
                         "Back Court Shot", "Back Court Shot", "Back Court Shot"]

    # Combine the new top level with the current column names
    multi_index = pd.MultiIndex.from_tuples(zip(top_level, team_stats.columns))

    # Apply the MultiIndex to the DataFrame
    team_stats.columns = multi_index

    # Drop the FGM, FGA, and FG% columns under the "Back Court Shot" header
    team_stats = team_stats.drop(columns=["Back Court Shot"], level=0)

    # Apply conditional formatting and ranking
    numeric_cols = [col for col in team_stats.columns if col[1] in ["FGM", "FGA", "FG%"]]
    df_with_ranks = team_stats.copy()

    # Add ranks
    for col in numeric_cols:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

    # Apply color formatting
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors
    
    # Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Filter for 'Raptors 905' team
raptors_905_df = df_with_ranks[df_with_ranks[(' ', 'TEAM')] == 'Raptors 905']

# Apply the styling only to 'Raptors 905' row
styled_raptors_df = raptors_905_df.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame for 'Raptors 905'
styled_raptors_df

In [22]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashteamshotlocations"

# Define the headers from the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters for the 8ft range
params = {
    "Conference": "",
    "DateFrom": "",
    "DateTo": "",
    "DistanceRange": "8ft Range",
    "Division": "",
    "GameScope": "",
    "GameSegment": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

# Verify if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    result_set = data["resultSets"]["rowSet"]
    columns = data["resultSets"]["headers"][1]["columnNames"]

    # Convert to a DataFrame
    team_stats = pd.DataFrame(result_set, columns=columns)

    # Drop the TEAM_ID column
    if "TEAM_ID" in team_stats.columns:
        team_stats = team_stats.drop(columns=["TEAM_ID"])

    # Rename FG_PCT columns to FG% and TEAM_NAME column to TEAM
    team_stats = team_stats.rename(columns=lambda x: x.replace("FG_PCT", "FG%") if "FG_PCT" in x else x)
    team_stats = team_stats.rename(columns={"TEAM_NAME": "TEAM"})

    # Add top-level column names to create a MultiIndex
    top_level = [" "] + ["Less Than 8ft.", "Less Than 8ft.", "Less Than 8ft.",
                         "8-16 ft.", "8-16 ft.", "8-16 ft.",
                         "16-24 ft.", "16-24 ft.", "16-24 ft.",
                         "24+ ft.", "24+ ft.", "24+ ft.",
                         "Back Court Shot", "Back Court Shot", "Back Court Shot"]

    # Combine the new top level with the current column names
    multi_index = pd.MultiIndex.from_tuples(zip(top_level, team_stats.columns))

    # Apply the MultiIndex to the DataFrame
    team_stats.columns = multi_index

    # Drop the FGM, FGA, and FG% columns under the "Back Court Shot" header
    team_stats = team_stats.drop(columns=["Back Court Shot"], level=0)

    # Apply conditional formatting and ranking
    numeric_cols = [col for col in team_stats.columns if col[1] in ["FGM", "FGA", "FG%"]]
    df_with_ranks = team_stats.copy()

    # Add ranks
    for col in numeric_cols:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

    # Apply color formatting
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors
    
    # Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Filter for 'opponent 905' team
opponent_905_df = df_with_ranks[df_with_ranks[(' ', 'TEAM')] == 'Westchester Knicks']

# Apply the styling only to 'opponent 905' row
styled_opponent_df = opponent_905_df.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame for 'opponent 905'
styled_opponent_df

In [18]:
team_stats.columns

Index(['TEAM_ID', 'TEAM_NAME', 'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT',
       'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT'],
      dtype='object')

In [31]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashteamshotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters for "By Zone"
params = {
    "Conference": "",
    "DateFrom": "",
    "DateTo": "",
    "DistanceRange": "By Zone",
    "Division": "",
    "GameScope": "",
    "GameSegment": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

# Verify if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    result_set = data["resultSets"]["rowSet"]
    columns = data["resultSets"]["headers"][1]["columnNames"]

    # Convert to a DataFrame
    team_stats = pd.DataFrame(result_set, columns=columns)

    # Drop the TEAM_ID column
    if "TEAM_ID" in team_stats.columns:
        team_stats = team_stats.drop(columns=["TEAM_ID"])

    # Rename FG_PCT columns to FG% and TEAM_NAME column to TEAM
    team_stats = team_stats.rename(columns=lambda x: x.replace("FG_PCT", "FG%") if "FG_PCT" in x else x)
    team_stats = team_stats.rename(columns={"TEAM_NAME": "TEAM"})

    # Drop extra columns to align with 19 columns
    team_stats = team_stats.iloc[:, :19]

    # Create top-level column names dynamically
    zones = [
        " ",  # For the TEAM column
        "RESTRICTED AREA", "RESTRICTED AREA", "RESTRICTED AREA",
        "IN THE PAINT (NON-RA)", "IN THE PAINT (NON-RA)", "IN THE PAINT (NON-RA)",
        "MID-RANGE", "MID-RANGE", "MID-RANGE",
        "Left Corner 3", "Left Corner 3", "Left Corner 3",
        "Right Corner 3", "Right Corner 3", "Right Corner 3",
        "Above the Break 3", "Above the Break 3", "Above the Break 3"
    ]

    lower_level = team_stats.columns.tolist()
    top_level = [zones[i] for i in range(len(lower_level))]

    # Pair top-level and lower-level columns
    multi_index = pd.MultiIndex.from_tuples(zip(top_level, lower_level))

    # Apply the MultiIndex to the DataFrame
    team_stats.columns = multi_index

    # Apply conditional formatting and ranking
    numeric_cols = [col for col in team_stats.columns if col[1] in ["FGM", "FGA", "FG%"]]
    df_with_ranks = team_stats.copy()

    # Add ranks
    for col in numeric_cols:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

    # Apply color formatting
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Style the DataFrame
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

In [30]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashteamshotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters for "By Zone"
params = {
    "Conference": "",
    "DateFrom": "",
    "DateTo": "",
    "DistanceRange": "By Zone",
    "Division": "",
    "GameScope": "",
    "GameSegment": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

# Verify if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    result_set = data["resultSets"]["rowSet"]
    columns = data["resultSets"]["headers"][1]["columnNames"]

    # Convert to a DataFrame
    team_stats = pd.DataFrame(result_set, columns=columns)

    # Drop the TEAM_ID column
    if "TEAM_ID" in team_stats.columns:
        team_stats = team_stats.drop(columns=["TEAM_ID"])

    # Rename FG_PCT columns to FG% and TEAM_NAME column to TEAM
    team_stats = team_stats.rename(columns=lambda x: x.replace("FG_PCT", "FG%") if "FG_PCT" in x else x)
    team_stats = team_stats.rename(columns={"TEAM_NAME": "TEAM"})

    # Drop extra columns to align with 19 columns
    team_stats = team_stats.iloc[:, :19]

    # Create top-level column names dynamically
    zones = [
        " ",  # For the TEAM column
        "RESTRICTED AREA", "RESTRICTED AREA", "RESTRICTED AREA",
        "IN THE PAINT (NON-RA)", "IN THE PAINT (NON-RA)", "IN THE PAINT (NON-RA)",
        "MID-RANGE", "MID-RANGE", "MID-RANGE",
        "Left Corner 3", "Left Corner 3", "Left Corner 3",
        "Right Corner 3", "Right Corner 3", "Right Corner 3",
        "Above the Break 3", "Above the Break 3", "Above the Break 3"
    ]

    lower_level = team_stats.columns.tolist()
    top_level = [zones[i] for i in range(len(lower_level))]

    # Pair top-level and lower-level columns
    multi_index = pd.MultiIndex.from_tuples(zip(top_level, lower_level))

    # Apply the MultiIndex to the DataFrame
    team_stats.columns = multi_index

    # Apply conditional formatting and ranking
    numeric_cols = [col for col in team_stats.columns if col[1] in ["FGM", "FGA", "FG%"]]
    df_with_ranks = team_stats.copy()

    # Add ranks
    for col in numeric_cols:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

    # Apply color formatting
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors
    
     # Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Filter for 'Raptors 905' team
raptors_905_df = df_with_ranks[df_with_ranks[(' ', 'TEAM')] == 'Raptors 905']

# Apply the styling only to 'Raptors 905' row
styled_raptors_df = raptors_905_df.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame for 'Raptors 905'
styled_raptors_df

In [32]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashteamshotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters for "By Zone"
params = {
    "Conference": "",
    "DateFrom": "",
    "DateTo": "",
    "DistanceRange": "By Zone",
    "Division": "",
    "GameScope": "",
    "GameSegment": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Base",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonSegment": "",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

# Verify if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    result_set = data["resultSets"]["rowSet"]
    columns = data["resultSets"]["headers"][1]["columnNames"]

    # Convert to a DataFrame
    team_stats = pd.DataFrame(result_set, columns=columns)

    # Drop the TEAM_ID column
    if "TEAM_ID" in team_stats.columns:
        team_stats = team_stats.drop(columns=["TEAM_ID"])

    # Rename FG_PCT columns to FG% and TEAM_NAME column to TEAM
    team_stats = team_stats.rename(columns=lambda x: x.replace("FG_PCT", "FG%") if "FG_PCT" in x else x)
    team_stats = team_stats.rename(columns={"TEAM_NAME": "TEAM"})

    # Drop extra columns to align with 19 columns
    team_stats = team_stats.iloc[:, :19]

    # Create top-level column names dynamically
    zones = [
        " ",  # For the TEAM column
        "RESTRICTED AREA", "RESTRICTED AREA", "RESTRICTED AREA",
        "IN THE PAINT (NON-RA)", "IN THE PAINT (NON-RA)", "IN THE PAINT (NON-RA)",
        "MID-RANGE", "MID-RANGE", "MID-RANGE",
        "Left Corner 3", "Left Corner 3", "Left Corner 3",
        "Right Corner 3", "Right Corner 3", "Right Corner 3",
        "Above the Break 3", "Above the Break 3", "Above the Break 3"
    ]

    lower_level = team_stats.columns.tolist()
    top_level = [zones[i] for i in range(len(lower_level))]

    # Pair top-level and lower-level columns
    multi_index = pd.MultiIndex.from_tuples(zip(top_level, lower_level))

    # Apply the MultiIndex to the DataFrame
    team_stats.columns = multi_index

    # Apply conditional formatting and ranking
    numeric_cols = [col for col in team_stats.columns if col[1] in ["FGM", "FGA", "FG%"]]
    df_with_ranks = team_stats.copy()

    # Add ranks
    for col in numeric_cols:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

    # Apply color formatting
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors
    
     # Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Filter for 'opponent 905' team
opponent_905_df = df_with_ranks[df_with_ranks[(' ', 'TEAM')] == 'Westchester Knicks']

# Apply the styling only to 'opponent 905' row
styled_opponent_df = opponent_905_df.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame for 'opponent 905'
styled_opponent_df

In [27]:
print("Columns in team_stats:", team_stats.columns)
print("Number of columns:", len(team_stats.columns))

Columns in team_stats: MultiIndex([(                    ' ', 'TEAM'),
            (      'RESTRICTED AREA',  'FGM'),
            (      'RESTRICTED AREA',  'FGA'),
            (      'RESTRICTED AREA',  'FG%'),
            (      'RESTRICTED AREA',  'FGM'),
            (      'RESTRICTED AREA',  'FGA'),
            (      'RESTRICTED AREA',  'FG%'),
            (      'RESTRICTED AREA',  'FGM'),
            (      'RESTRICTED AREA',  'FGA'),
            ('IN THE PAINT (NON-RA)',  'FG%'),
            ('IN THE PAINT (NON-RA)',  'FGM'),
            ('IN THE PAINT (NON-RA)',  'FGA'),
            ('IN THE PAINT (NON-RA)',  'FG%'),
            ('IN THE PAINT (NON-RA)',  'FGM'),
            ('IN THE PAINT (NON-RA)',  'FGA'),
            ('IN THE PAINT (NON-RA)',  'FG%'),
            ('IN THE PAINT (NON-RA)',  'FGM'),
            ('IN THE PAINT (NON-RA)',  'FGA'),
            (            'MID-RANGE',  'FG%'),
            (            'MID-RANGE',  'FGM'),
            (            'MID-RANGE',

In [16]:
team_stats.columns

Index(['TEAM_ID', 'TEAM_NAME', 'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT',
       'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT',
       'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT'],
      dtype='object')

In [17]:
print(team_stats.columns.tolist())

['TEAM_ID', 'TEAM_NAME', 'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT', 'FGM', 'FGA', 'FG_PCT']


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/opponent-shooting/"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# --- New Code Integration: Drop Specific MultiIndex Levels ---
levels_to_drop = [1, 2, 4, 5]  # Specify the level indices to drop (1-based indexing)

# Check if the columns are MultiIndex
if isinstance(team_stats.columns, pd.MultiIndex):
    # Drop the specified levels
    team_stats.columns = team_stats.columns.droplevel(levels_to_drop)
    #print("Dropped levels:", levels_to_drop)
    
    # Replace 'Unnamed: 0_level_0' with 'TEAM' in MultiIndex
    new_columns = []
    for col in team_stats.columns:
        if col[0] == 'Unnamed: 0_level_0':
            new_columns.append((' ', col[1]))
        else:
            new_columns.append(col)
    team_stats.columns = pd.MultiIndex.from_tuples(new_columns)
    #print("Updated columns after replacing 'Unnamed: 0_level_0' with 'TEAM':")
else:
    print("Columns are not MultiIndex. No levels to drop.")

# Drop rows with NaN values
team_stats = team_stats.dropna()

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns.tolist()

# Handle FG% columns: convert to numeric if they contain strings
for col in team_stats.columns:
    if 'FG%' in col and col not in numeric_cols:
        team_stats[col] = pd.to_numeric(team_stats[col], errors='coerce')
        numeric_cols.append(col)

# Re-check numeric columns after converting FG% columns
numeric_cols = team_stats.select_dtypes(include=['number']).columns.tolist()

# Calculate ranks and append ranks to numeric columns for display
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (all in descending order since no inversion is needed)
    ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        # Inverted normalization for ascending rank
        norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame
styled_df

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/opponent-shooting/"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# --- New Code Integration: Drop Specific MultiIndex Levels ---
levels_to_drop = [1, 2, 4, 5]  # Specify the level indices to drop (1-based indexing)

# Check if the columns are MultiIndex
if isinstance(team_stats.columns, pd.MultiIndex):
    # Drop the specified levels
    team_stats.columns = team_stats.columns.droplevel(levels_to_drop)
    #print("Dropped levels:", levels_to_drop)
    
    # Replace 'Unnamed: 0_level_0' with 'TEAM' in MultiIndex
    new_columns = []
    for col in team_stats.columns:
        if col[0] == 'Unnamed: 0_level_0':
            new_columns.append((' ', col[1]))
        else:
            new_columns.append(col)
    team_stats.columns = pd.MultiIndex.from_tuples(new_columns)
    #print("Updated columns after replacing 'Unnamed: 0_level_0' with 'TEAM':")
else:
    print("Columns are not MultiIndex. No levels to drop.")

# Drop rows with NaN values
team_stats = team_stats.dropna()

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns.tolist()

# Handle FG% columns: convert to numeric if they contain strings
for col in team_stats.columns:
    if 'FG%' in col and col not in numeric_cols:
        team_stats[col] = pd.to_numeric(team_stats[col], errors='coerce')
        numeric_cols.append(col)

# Re-check numeric columns after converting FG% columns
numeric_cols = team_stats.select_dtypes(include=['number']).columns.tolist()

# Calculate ranks and append ranks to numeric columns for display
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (all in descending order since no inversion is needed)
    ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        # Inverted normalization for ascending rank
        norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Filter for 'Raptors 905' team
raptors_905_df = df_with_ranks[df_with_ranks[(' ', 'Team')] == 'Raptors 905']

# Apply the styling only to 'Raptors 905' row
styled_raptors_df = raptors_905_df.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame for 'Raptors 905'
styled_raptors_df

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

# Mac file path
# "/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

# PC file path
# r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/teams/opponent-shooting/"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
team_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

# --- New Code Integration: Drop Specific MultiIndex Levels ---
levels_to_drop = [1, 2, 4, 5]  # Specify the level indices to drop (1-based indexing)

# Check if the columns are MultiIndex
if isinstance(team_stats.columns, pd.MultiIndex):
    # Drop the specified levels
    team_stats.columns = team_stats.columns.droplevel(levels_to_drop)
    #print("Dropped levels:", levels_to_drop)
    
    # Replace 'Unnamed: 0_level_0' with 'TEAM' in MultiIndex
    new_columns = []
    for col in team_stats.columns:
        if col[0] == 'Unnamed: 0_level_0':
            new_columns.append((' ', col[1]))
        else:
            new_columns.append(col)
    team_stats.columns = pd.MultiIndex.from_tuples(new_columns)
    #print("Updated columns after replacing 'Unnamed: 0_level_0' with 'TEAM':")
else:
    print("Columns are not MultiIndex. No levels to drop.")

# Drop rows with NaN values
team_stats = team_stats.dropna()

# Ensure numeric columns are correctly identified
numeric_cols = team_stats.select_dtypes(include=['number']).columns.tolist()

# Handle FG% columns: convert to numeric if they contain strings
for col in team_stats.columns:
    if 'FG%' in col and col not in numeric_cols:
        team_stats[col] = pd.to_numeric(team_stats[col], errors='coerce')
        numeric_cols.append(col)

# Re-check numeric columns after converting FG% columns
numeric_cols = team_stats.select_dtypes(include=['number']).columns.tolist()

# Calculate ranks and append ranks to numeric columns for display
df_with_ranks = team_stats.copy()

for col in numeric_cols:
    # Calculate ranks (all in descending order since no inversion is needed)
    ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
    
    # Append ranks to column values for display
    df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

# Color coding logic
col_min = team_stats[numeric_cols].min()
col_max = team_stats[numeric_cols].max()

def luminance(color):
    rgb = mcolors.hex2color(color)
    return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

def color_cells(row):
    colors = []
    for col in numeric_cols:
        value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
        # Inverted normalization for ascending rank
        norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
        color = mcolors.to_hex(plt.cm.RdYlGn(norm))
        text_color = 'white' if luminance(color) < 0.5 else 'black'
        colors.append(f"background-color: {color}; color: {text_color};")
    return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Filter for 'opponent 905' team
opponent_905_df = df_with_ranks[df_with_ranks[(' ', 'Team')] == 'Westchester Knicks']

# Apply the styling only to 'opponent 905' row
styled_opponent_df = opponent_905_df.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame for 'opponent 905'
styled_opponent_df

In [26]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashteamshotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "Conference": "",
    "DateFrom": "",
    "DateTo": "",
    "DistanceRange": "8ft Range",
    "Division": "",
    "GameScope": "",
    "GameSegment": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Opponent",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract column names
        column_names = headers[1]["columnNames"]

        # Convert to a DataFrame
        team_stats = pd.DataFrame(result_set, columns=column_names)

        # Drop the TEAM_ID column
        if "TEAM_ID" in team_stats.columns:
            team_stats = team_stats.drop(columns=["TEAM_ID"])

        # Rename OPP_FGM, OPP_FGA, and OPP_FG% to FGM, FGA, and FG%
        team_stats = team_stats.rename(columns=lambda x: x.replace("OPP_FGM", "FGM")
                                                         .replace("OPP_FGA", "FGA")
                                                         .replace("OPP_FG_PCT", "FG%"))
        team_stats = team_stats.rename(columns={"TEAM_NAME": "TEAM"})

        # Add top-level column names to create a MultiIndex
        top_level = [" "] + ["Less Than 8 ft.", "Less Than 8 ft.", "Less Than 8 ft.",
                             "8-16 ft.", "8-16 ft.", "8-16 ft.",
                             "16-24 ft.", "16-24 ft.", "16-24 ft.",
                             "24+ ft.", "24+ ft.", "24+ ft.",
                             "Back Court Shot", "Back Court Shot", "Back Court Shot"]

        if len(top_level) != len(team_stats.columns):
            raise ValueError("Mismatch between top-level column structure and actual data columns.")

        # Combine the new top-level columns with the current column names
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, team_stats.columns))
        team_stats.columns = multi_index

        # Drop the "Back Court Shot" columns
        team_stats = team_stats.drop(columns=["Back Court Shot"], level=0)

        # Apply conditional formatting and ranking
        numeric_cols = [col for col in team_stats.columns if col[1] in ["FGM", "FGA", "FG%"]]
        df_with_ranks = team_stats.copy()

        for col in numeric_cols:
            # Set ascending=True for ascending order ranks
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
            df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

        col_min = team_stats[numeric_cols].min()
        col_max = team_stats[numeric_cols].max()

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                value = team_stats.loc[row.name, col]
                # Reverse normalization for ascending order
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

    else:
        print("No valid data found in the response.")
else:
    print(f"Request failed with status code {response.status_code}")

In [27]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashteamshotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "Conference": "",
    "DateFrom": "",
    "DateTo": "",
    "DistanceRange": "8ft Range",
    "Division": "",
    "GameScope": "",
    "GameSegment": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Opponent",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract column names
        column_names = headers[1]["columnNames"]

        # Convert to a DataFrame
        team_stats = pd.DataFrame(result_set, columns=column_names)

        # Drop the TEAM_ID column
        if "TEAM_ID" in team_stats.columns:
            team_stats = team_stats.drop(columns=["TEAM_ID"])

        # Rename OPP_FGM, OPP_FGA, and OPP_FG% to FGM, FGA, and FG%
        team_stats = team_stats.rename(columns=lambda x: x.replace("OPP_FGM", "FGM")
                                                         .replace("OPP_FGA", "FGA")
                                                         .replace("OPP_FG_PCT", "FG%"))
        team_stats = team_stats.rename(columns={"TEAM_NAME": "TEAM"})

        # Add top-level column names to create a MultiIndex
        top_level = [" "] + ["Less Than 8 ft.", "Less Than 8 ft.", "Less Than 8 ft.",
                             "8-16 ft.", "8-16 ft.", "8-16 ft.",
                             "16-24 ft.", "16-24 ft.", "16-24 ft.",
                             "24+ ft.", "24+ ft.", "24+ ft.",
                             "Back Court Shot", "Back Court Shot", "Back Court Shot"]

        if len(top_level) != len(team_stats.columns):
            raise ValueError("Mismatch between top-level column structure and actual data columns.")

        # Combine the new top-level columns with the current column names
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, team_stats.columns))
        team_stats.columns = multi_index

        # Drop the "Back Court Shot" columns
        team_stats = team_stats.drop(columns=["Back Court Shot"], level=0)

        # Apply conditional formatting and ranking
        numeric_cols = [col for col in team_stats.columns if col[1] in ["FGM", "FGA", "FG%"]]
        df_with_ranks = team_stats.copy()

        for col in numeric_cols:
            # Set ascending=True for ascending order ranks
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
            df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

        col_min = team_stats[numeric_cols].min()
        col_max = team_stats[numeric_cols].max()

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                value = team_stats.loc[row.name, col]
                # Reverse normalization for ascending order
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Filter for 'Raptors 905' team
raptors_905_df = df_with_ranks[df_with_ranks[(' ', 'TEAM')] == 'Raptors 905']

# Apply the styling only to 'Raptors 905' row
styled_raptors_df = raptors_905_df.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame for 'Raptors 905'
styled_raptors_df

In [28]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashteamshotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "Conference": "",
    "DateFrom": "",
    "DateTo": "",
    "DistanceRange": "8ft Range",
    "Division": "",
    "GameScope": "",
    "GameSegment": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Opponent",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract column names
        column_names = headers[1]["columnNames"]

        # Convert to a DataFrame
        team_stats = pd.DataFrame(result_set, columns=column_names)

        # Drop the TEAM_ID column
        if "TEAM_ID" in team_stats.columns:
            team_stats = team_stats.drop(columns=["TEAM_ID"])

        # Rename OPP_FGM, OPP_FGA, and OPP_FG% to FGM, FGA, and FG%
        team_stats = team_stats.rename(columns=lambda x: x.replace("OPP_FGM", "FGM")
                                                         .replace("OPP_FGA", "FGA")
                                                         .replace("OPP_FG_PCT", "FG%"))
        team_stats = team_stats.rename(columns={"TEAM_NAME": "TEAM"})

        # Add top-level column names to create a MultiIndex
        top_level = [" "] + ["Less Than 8 ft.", "Less Than 8 ft.", "Less Than 8 ft.",
                             "8-16 ft.", "8-16 ft.", "8-16 ft.",
                             "16-24 ft.", "16-24 ft.", "16-24 ft.",
                             "24+ ft.", "24+ ft.", "24+ ft.",
                             "Back Court Shot", "Back Court Shot", "Back Court Shot"]

        if len(top_level) != len(team_stats.columns):
            raise ValueError("Mismatch between top-level column structure and actual data columns.")

        # Combine the new top-level columns with the current column names
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, team_stats.columns))
        team_stats.columns = multi_index

        # Drop the "Back Court Shot" columns
        team_stats = team_stats.drop(columns=["Back Court Shot"], level=0)

        # Apply conditional formatting and ranking
        numeric_cols = [col for col in team_stats.columns if col[1] in ["FGM", "FGA", "FG%"]]
        df_with_ranks = team_stats.copy()

        for col in numeric_cols:
            # Set ascending=True for ascending order ranks
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
            df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

        col_min = team_stats[numeric_cols].min()
        col_max = team_stats[numeric_cols].max()

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                value = team_stats.loc[row.name, col]
                # Reverse normalization for ascending order
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

# Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Filter for 'opponent 905' team
opponent_905_df = df_with_ranks[df_with_ranks[(' ', 'TEAM')] == 'Westchester Knicks']

# Apply the styling only to 'opponent 905' row
styled_opponent_df = opponent_905_df.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame for 'opponent 905'
styled_opponent_df

In [30]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashteamshotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "Conference": "",
    "DateFrom": "",
    "DateTo": "",
    "DistanceRange": "By Zone",
    "Division": "",
    "GameScope": "",
    "GameSegment": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Opponent",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract column names
        column_names = headers[1]["columnNames"]

        # Convert to a DataFrame
        team_stats = pd.DataFrame(result_set, columns=column_names)

        # Drop the TEAM_ID column
        if "TEAM_ID" in team_stats.columns:
            team_stats = team_stats.drop(columns=["TEAM_ID"])

        # Rename OPP_FGM, OPP_FGA, and OPP_FG% to FGM, FGA, and FG%
        team_stats = team_stats.rename(columns=lambda x: x.replace("OPP_FGM", "FGM")
                                                         .replace("OPP_FGA", "FGA")
                                                         .replace("OPP_FG_PCT", "FG%"))
        team_stats = team_stats.rename(columns={"TEAM_NAME": "TEAM"})

        # Add top-level column names to create a MultiIndex
        top_level = [" "] + ["Restricted Area", "Restricted Area", "Restricted Area",
                             "In The Paint (Non-RA)", "In The Paint (Non-RA)", "In The Paint (Non-RA)",
                             "Mid-Range", "Mid-Range", "Mid-Range",
                             "Left Corner 3", "Left Corner 3", "Left Corner 3",
                             "Right Corner 3", "Right Corner 3", "Right Corner 3",
                             "Above the Break 3", "Above the Break 3", "Above the Break 3",
                             "Backcourt", "Backcourt", "Backcourt",
                             "Corner 3", "Corner 3", "Corner 3"]

        if len(top_level) != len(team_stats.columns):
            raise ValueError("Mismatch between top-level column structure and actual data columns.")

        # Combine the new top-level columns with the current column names
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, team_stats.columns))
        team_stats.columns = multi_index

        # Drop the "Backcourt" columns
        team_stats = team_stats.drop(columns=["Backcourt"], level=0)

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in team_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in team_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        team_stats = team_stats[column_order]

        # Apply conditional formatting and ranking
        numeric_cols = [col for col in team_stats.columns if col[1] in ["FGM", "FGA", "FG%"]]
        df_with_ranks = team_stats.copy()

        for col in numeric_cols:
            # Set ascending=True for ascending order ranks
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
            df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

        col_min = team_stats[numeric_cols].min()
        col_max = team_stats[numeric_cols].max()

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                value = team_stats.loc[row.name, col]
                # Reverse normalization for ascending order
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

    else:
        print("No valid data found in the response.")
else:
    print(f"Request failed with status code {response.status_code}")

In [31]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashteamshotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "Conference": "",
    "DateFrom": "",
    "DateTo": "",
    "DistanceRange": "By Zone",
    "Division": "",
    "GameScope": "",
    "GameSegment": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Opponent",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract column names
        column_names = headers[1]["columnNames"]

        # Convert to a DataFrame
        team_stats = pd.DataFrame(result_set, columns=column_names)

        # Drop the TEAM_ID column
        if "TEAM_ID" in team_stats.columns:
            team_stats = team_stats.drop(columns=["TEAM_ID"])

        # Rename OPP_FGM, OPP_FGA, and OPP_FG% to FGM, FGA, and FG%
        team_stats = team_stats.rename(columns=lambda x: x.replace("OPP_FGM", "FGM")
                                                         .replace("OPP_FGA", "FGA")
                                                         .replace("OPP_FG_PCT", "FG%"))
        team_stats = team_stats.rename(columns={"TEAM_NAME": "TEAM"})

        # Add top-level column names to create a MultiIndex
        top_level = [" "] + ["Restricted Area", "Restricted Area", "Restricted Area",
                             "In The Paint (Non-RA)", "In The Paint (Non-RA)", "In The Paint (Non-RA)",
                             "Mid-Range", "Mid-Range", "Mid-Range",
                             "Left Corner 3", "Left Corner 3", "Left Corner 3",
                             "Right Corner 3", "Right Corner 3", "Right Corner 3",
                             "Above the Break 3", "Above the Break 3", "Above the Break 3",
                             "Backcourt", "Backcourt", "Backcourt",
                             "Corner 3", "Corner 3", "Corner 3"]

        if len(top_level) != len(team_stats.columns):
            raise ValueError("Mismatch between top-level column structure and actual data columns.")

        # Combine the new top-level columns with the current column names
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, team_stats.columns))
        team_stats.columns = multi_index

        # Drop the "Backcourt" columns
        team_stats = team_stats.drop(columns=["Backcourt"], level=0)

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in team_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in team_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        team_stats = team_stats[column_order]

        # Apply conditional formatting and ranking
        numeric_cols = [col for col in team_stats.columns if col[1] in ["FGM", "FGA", "FG%"]]
        df_with_ranks = team_stats.copy()

        for col in numeric_cols:
            # Set ascending=True for ascending order ranks
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
            df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

        col_min = team_stats[numeric_cols].min()
        col_max = team_stats[numeric_cols].max()

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                value = team_stats.loc[row.name, col]
                # Reverse normalization for ascending order
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Filter for 'Raptors 905' team
raptors_905_df = df_with_ranks[df_with_ranks[(' ', 'TEAM')] == 'Raptors 905']

# Apply the styling only to 'Raptors 905' row
styled_raptors_df = raptors_905_df.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame for 'Raptors 905'
styled_raptors_df

In [32]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashteamshotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "Conference": "",
    "DateFrom": "",
    "DateTo": "",
    "DistanceRange": "By Zone",
    "Division": "",
    "GameScope": "",
    "GameSegment": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Opponent",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract column names
        column_names = headers[1]["columnNames"]

        # Convert to a DataFrame
        team_stats = pd.DataFrame(result_set, columns=column_names)

        # Drop the TEAM_ID column
        if "TEAM_ID" in team_stats.columns:
            team_stats = team_stats.drop(columns=["TEAM_ID"])

        # Rename OPP_FGM, OPP_FGA, and OPP_FG% to FGM, FGA, and FG%
        team_stats = team_stats.rename(columns=lambda x: x.replace("OPP_FGM", "FGM")
                                                         .replace("OPP_FGA", "FGA")
                                                         .replace("OPP_FG_PCT", "FG%"))
        team_stats = team_stats.rename(columns={"TEAM_NAME": "TEAM"})

        # Add top-level column names to create a MultiIndex
        top_level = [" "] + ["Restricted Area", "Restricted Area", "Restricted Area",
                             "In The Paint (Non-RA)", "In The Paint (Non-RA)", "In The Paint (Non-RA)",
                             "Mid-Range", "Mid-Range", "Mid-Range",
                             "Left Corner 3", "Left Corner 3", "Left Corner 3",
                             "Right Corner 3", "Right Corner 3", "Right Corner 3",
                             "Above the Break 3", "Above the Break 3", "Above the Break 3",
                             "Backcourt", "Backcourt", "Backcourt",
                             "Corner 3", "Corner 3", "Corner 3"]

        if len(top_level) != len(team_stats.columns):
            raise ValueError("Mismatch between top-level column structure and actual data columns.")

        # Combine the new top-level columns with the current column names
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, team_stats.columns))
        team_stats.columns = multi_index

        # Drop the "Backcourt" columns
        team_stats = team_stats.drop(columns=["Backcourt"], level=0)

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in team_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in team_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        team_stats = team_stats[column_order]

        # Apply conditional formatting and ranking
        numeric_cols = [col for col in team_stats.columns if col[1] in ["FGM", "FGA", "FG%"]]
        df_with_ranks = team_stats.copy()

        for col in numeric_cols:
            # Set ascending=True for ascending order ranks
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
            df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

        col_min = team_stats[numeric_cols].min()
        col_max = team_stats[numeric_cols].max()

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                value = team_stats.loc[row.name, col]
                # Reverse normalization for ascending order
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply the coloring logic to the DataFrame with ranks
styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Filter for 'opponent 905' team
opponent_905_df = df_with_ranks[df_with_ranks[(' ', 'TEAM')] == 'Westchester Knicks']

# Apply the styling only to 'opponent 905' row
styled_opponent_df = opponent_905_df.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

# Display the styled DataFrame for 'opponent 905'
styled_opponent_df

# Player Stats

In [30]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# API URL
url = "https://stats.gleague.nba.com/stats/leagueLeaders"
params = {
    "LeagueID": "20",
    "PerMode": "PerGame",
    "StatCategory": "PTS",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "Scope": "S"
}

# Headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Connection": "keep-alive"
}

# Fetching the data
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extracting headers and rows
    headers = data['resultSet']['headers']
    rows = data['resultSet']['rowSet']

    # Creating a DataFrame
    stats = pd.DataFrame(rows, columns=headers)

    # Drop unnecessary columns
    columns_to_drop = ["PLAYER_ID", "RANK", "TEAM_ID", "GP"]
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    stats = stats[stats["MIN"] >= 15]

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns (after filtering)
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col == "TOV":
            # Invert ranking for PF (lower is better)
            ranks = stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = stats[col].astype(str) + ' (' + ranks.astype(str) + ')'
        ranks_data[col] = ranks  # Store ranks for color mapping

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank = ranks_data[col].loc[row.name]
            max_rank = ranks_data[col].max()
            min_rank = ranks_data[col].min()
            # Normalize rank to 0-1 scale for color mapping
            norm = (rank - min_rank) / (max_rank - min_rank) if max_rank != min_rank else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert to make 1 (best) green
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

,PLAYER,TEAM,MIN,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,EFF
0,Bryce McGowens,RCR,35.3,9.6 (3),19.6 (5),0.491 (84),2.5 (32),7.2 (32),0.342 (117),4.5 (1),5.1 (1),0.893 (39),0.9 (169),2.5 (200),3.5 (202),3.3 (74),1.5 (36),0.7 (96),3.6 (259),29.4 (1),24.1 (15)
1,Brandon Williams,TEX,37.9,9.9 (2),20.8 (2),0.476 (102),3.3 (12),8.4 (12),0.388 (67),3.3 (6),4.9 (2),0.667 (188),1.8 (73),3.9 (108),5.6 (99),6.8 (16),2.0 (15),0.8 (76),4.6 (270),28.5 (2),26.5 (7)
2,JD Davison,MNE,35.0,9.4 (4),20.2 (3),0.467 (115),1.8 (100),5.8 (87),0.314 (159),3.8 (3),4.9 (2),0.78 (110),1.3 (116),3.7 (121),5.0 (120),8.8 (3),1.5 (36),1.3 (29),2.9 (236),27.3 (3),29.0 (2)
3,T.J. Warren,WES,36.6,10.6 (1),20.9 (1),0.507 (62),2.8 (21),7.7 (23),0.364 (86),1.8 (52),2.4 (52),0.75 (134),3.9 (4),4.2 (90),8.1 (27),3.5 (67),1.9 (18),0.7 (96),2.4 (206),27.2 (4),28.1 (4)
4,Josh Christopher,SXF,35.7,9.3 (5),20.0 (4),0.467 (115),3.1 (16),9.8 (6),0.316 (156),1.8 (52),2.1 (67),0.84 (71),0.9 (169),5.2 (45),6.1 (81),4.5 (38),2.4 (7),0.7 (96),3.1 (244),24.6 (5),24.2 (13)
5,Keshad Johnson,SXF,35.1,8.7 (9),15.1 (32),0.574 (26),1.6 (115),3.9 (145),0.4 (58),3.0 (11),3.7 (13),0.818 (85),3.0 (18),5.8 (30),8.8 (15),1.4 (204),1.2 (73),0.8 (76),1.3 (75),24.3 (6),28.1 (4)
6,Isaiah Wong,GBO,31.4,7.9 (16),17.0 (16),0.463 (122),2.0 (74),4.9 (115),0.41 (47),4.1 (2),4.9 (2),0.846 (65),1.3 (116),2.6 (195),3.9 (172),3.6 (63),1.5 (36),0.4 (151),2.5 (212),24.1 (7),21.1 (35)
7,A.J. Lawson,LIN,35.1,7.7 (22),17.6 (13),0.438 (156),2.5 (32),7.9 (18),0.322 (143),3.1 (9),3.7 (13),0.829 (77),0.9 (169),5.7 (33),6.6 (73),2.5 (123),1.2 (73),0.3 (169),1.6 (122),24.0 (8),22.5 (21)
8,Cole Swider,MCC,33.1,8.3 (10),16.0 (23),0.516 (55),5.4 (1),10.6 (2),0.506 (9),1.0 (138),1.0 (186),1.0 (1),1.1 (140),4.4 (81),5.5 (104),2.4 (128),1.5 (36),0.5 (126),1.5 (108),24.0 (8),24.6 (10)
9,Jeff Dowtin Jr.,DEL,33.3,9.3 (5),18.3 (9),0.507 (62),2.5 (32),6.0 (76),0.417 (38),1.9 (45),2.3 (55),0.833 (73),0.3 (251),2.8 (178),3.0 (228),4.6 (37),1.1 (89),0.5 (126),1.6 (122),23.6 (10),21.9 (27)


In [34]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# API URL
url = "https://stats.gleague.nba.com/stats/leagueLeaders"
params = {
    "LeagueID": "20",
    "PerMode": "PerGame",
    "StatCategory": "PTS",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "Scope": "S"
}

# Headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Connection": "keep-alive"
}

# Fetching the data
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extracting headers and rows
    headers = data['resultSet']['headers']
    rows = data['resultSet']['rowSet']

    # Creating a DataFrame
    stats = pd.DataFrame(rows, columns=headers)

    # Drop unnecessary columns
    columns_to_drop = ["PLAYER_ID", "RANK", "TEAM_ID", "GP"]
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    filtered_stats = stats[stats["MIN"] >= 15]

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns based on all players who play at least 15 minutes
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col == "TOV":
            # Inverted ranking for PF (lower is better)
            ranks = filtered_stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = filtered_stats[col].rank(ascending=False, method='min').astype(int)
        # Remove parentheses for players not meeting the threshold
        df_with_ranks[col] = stats[col].astype(str) + df_with_ranks.index.map(
            lambda idx: f" ({ranks[idx]})" if idx in ranks.index else ""
        )
        ranks_data[col] = ranks

    # Compute color scaling based on ranks
    rank_min = 1
    rank_max = filtered_stats.shape[0]  # Maximum rank is total filtered players

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank_str = row[col]
            if " (" not in rank_str:  # No rank for players below the threshold
                colors.append("background-color: none; color: black;")
                continue
            rank = int(rank_str.split("(")[-1].replace(")", ""))  # Extract the rank from the string
            # Normalize rank for color mapping
            norm = (rank - rank_min) / (rank_max - rank_min) if rank_max > rank_min else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert scale for ranks (lower is better)
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply RAP team filter for display
    stats = stats[stats["TEAM"] == "RAP"]
    df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "RAP"]

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

,PLAYER,TEAM,MIN,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,EFF
34,Kennedy Chandler,RAP,31.3,8.2 (11),15.5 (29),0.527 (45),1.3 (142),3.1 (179),0.432 (30),1.1 (126),2.2 (59),0.5 (250),1.3 (116),2.4 (210),3.8 (178),7.5 (7),1.9 (18),0.6 (113),2.3 (194),19.2 (35),22.3 (24)
36,Eugene Omoruyi,RAP,27.3,7.6 (23),14.4 (41),0.525 (47),1.1 (164),4.0 (139),0.286 (190),1.6 (64),3.1 (27),0.5 (250),2.6 (26),5.7 (33),8.3 (25),0.9 (238),0.7 (186),0.6 (113),3.7 (261),19.0 (36),17.3 (79)
91,Quincy Guerrier,RAP,33.2,6.0 (67),11.0 (112),0.545 (37),0.8 (195),3.1 (179),0.25 (222),1.1 (126),1.7 (107),0.667 (188),1.8 (73),5.1 (48),6.9 (57),1.1 (227),0.6 (202),0.1 (242),1.3 (74),14.7 (91),16.4 (96)
141,Charlie Brown Jr.,RAP,31.4,3.9 (156),10.6 (124),0.368 (240),2.2 (57),6.6 (53),0.339 (122),1.0 (139),1.2 (167),0.818 (86),0.8 (184),2.1 (233),2.9 (234),2.2 (142),3.8 (1),1.0 (53),2.3 (194),12.0 (142),12.7 (162)
187,Dylan Disu,RAP,20.2,3.5 (185),6.8 (211),0.519 (52),1.7 (109),4.0 (139),0.417 (41),0.5 (205),0.7 (214),0.75 (133),0.9 (169),3.6 (131),4.5 (143),1.5 (200),1.1 (89),1.0 (53),1.8 (150),9.5 (187),12.4 (176)
194,Tyreke Key,RAP,26.8,3.4 (195),7.5 (194),0.456 (135),1.9 (89),4.6 (118),0.418 (40),0.4 (228),0.7 (214),0.625 (216),0.9 (169),1.8 (250),2.7 (240),2.0 (158),0.8 (152),0.5 (126),0.8 (19),9.3 (191),10.0 (216)
211,Evan Gilyard,RAP,18.3,3.5 (185),8.1 (173),0.427 (170),1.0 (179),3.8 (149),0.262 (213),0.5 (205),0.6 (228),0.857 (58),0.3 (251),1.7 (255),2.0 (260),4.1 (48),0.7 (186),0.1 (242),1.5 (109),8.6 (210),9.3 (226)
248,Ulrich Chomche,RAP,21.5,2.9 (218),5.4 (243),0.537 (43),0.0 (256),0.1 (260),0.0 (256),0.3 (241),0.6 (228),0.5 (250),2.1 (54),4.0 (103),6.1 (81),0.6 (258),0.1 (266),2.9 (5),1.6 (123),6.4 (247),11.7 (186)
270,Tylor Perry,RAP,13.1,1.0,2.9,0.346,0.9,2.2,0.4,1.0,1.1,0.9,0.0,1.1,1.1,1.7,0.6,0.2,0.4,4.8,5.9
290,Kevin Obanor,RAP,9.9,1.4,2.7,0.519,0.2,0.8,0.25,0.4,0.9,0.444,1.0,1.6,2.6,0.3,0.1,0.0,0.5,3.5,4.2


In [35]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# API URL
url = "https://stats.gleague.nba.com/stats/leagueLeaders"
params = {
    "LeagueID": "20",
    "PerMode": "PerGame",
    "StatCategory": "PTS",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "Scope": "S"
}

# Headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Connection": "keep-alive"
}

# Fetching the data
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extracting headers and rows
    headers = data['resultSet']['headers']
    rows = data['resultSet']['rowSet']

    # Creating a DataFrame
    stats = pd.DataFrame(rows, columns=headers)

    # Drop unnecessary columns
    columns_to_drop = ["PLAYER_ID", "RANK", "TEAM_ID", "GP"]
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    filtered_stats = stats[stats["MIN"] >= 15]

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns based on all players who play at least 15 minutes
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col == "TOV":
            # Inverted ranking for PF (lower is better)
            ranks = filtered_stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = filtered_stats[col].rank(ascending=False, method='min').astype(int)
        # Remove parentheses for players not meeting the threshold
        df_with_ranks[col] = stats[col].astype(str) + df_with_ranks.index.map(
            lambda idx: f" ({ranks[idx]})" if idx in ranks.index else ""
        )
        ranks_data[col] = ranks

    # Compute color scaling based on ranks
    rank_min = 1
    rank_max = filtered_stats.shape[0]  # Maximum rank is total filtered players

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank_str = row[col]
            if " (" not in rank_str:  # No rank for players below the threshold
                colors.append("background-color: none; color: black;")
                continue
            rank = int(rank_str.split("(")[-1].replace(")", ""))  # Extract the rank from the string
            # Normalize rank for color mapping
            norm = (rank - rank_min) / (rank_max - rank_min) if rank_max > rank_min else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert scale for ranks (lower is better)
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply RAP team filter for display
    stats = stats[stats["TEAM"] == "DEL"]
    df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "DEL"]

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

,PLAYER,TEAM,MIN,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,EFF
9,Jeff Dowtin Jr.,DEL,33.3,9.3 (5),18.3 (9),0.507 (62),2.5 (32),6.0 (77),0.417 (41),1.9 (45),2.3 (54),0.833 (74),0.3 (251),2.8 (178),3.0 (228),4.6 (37),1.1 (89),0.5 (126),1.6 (123),23.6 (10),21.9 (27)
42,Darius Bazley,DEL,30.8,6.4 (51),12.8 (71),0.496 (77),1.6 (115),4.5 (119),0.367 (80),2.5 (24),3.6 (15),0.675 (187),2.3 (45),7.4 (5),9.6 (7),1.8 (173),1.5 (36),1.5 (22),2.5 (212),18.5 (43),22.8 (20)
46,Pete Nance,DEL,34.2,6.3 (54),13.1 (63),0.483 (96),2.1 (69),6.2 (65),0.339 (122),1.7 (59),2.3 (54),0.714 (165),1.8 (73),5.0 (50),6.8 (62),3.3 (74),0.8 (152),2.0 (12),1.9 (162),17.9 (47),21.4 (32)
79,Justin Edwards,DEL,32.7,5.9 (71),13.3 (56),0.445 (151),2.3 (46),6.5 (55),0.352 (109),1.3 (93),1.5 (124),0.875 (46),1.0 (150),2.5 (199),3.5 (202),1.7 (184),1.0 (114),0.5 (126),2.5 (212),15.7 (80),12.5 (169)
116,Judah Mintz,DEL,24.5,3.8 (161),8.8 (158),0.43 (166),0.8 (195),2.5 (204),0.313 (159),3.0 (11),3.5 (16),0.867 (53),0.2 (265),2.4 (210),2.6 (244),3.8 (55),0.8 (152),0.2 (216),2.6 (219),13.1 (117),12.5 (169)
164,Isaiah Mobley,DEL,25.9,3.6 (176),10.7 (121),0.339 (258),1.3 (142),4.0 (139),0.318 (147),1.1 (126),2.1 (67),0.522 (248),1.5 (99),6.1 (25),7.6 (39),3.1 (87),0.5 (219),0.9 (66),1.4 (99),10.7 (165),13.5 (149)
233,Marcus Bagley,DEL,22.2,2.8 (223),6.5 (217),0.435 (158),0.8 (195),2.8 (194),0.278 (204),0.4 (228),0.7 (214),0.556 (240),1.7 (82),3.5 (134),5.2 (118),1.1 (227),0.6 (202),0.3 (169),1.0 (39),7.2 (233),9.5 (222)
236,Jarron Cumberland,DEL,22.0,2.0 (256),5.2 (251),0.386 (224),0.7 (203),2.9 (191),0.25 (222),1.6 (64),2.1 (67),0.783 (109),1.0 (150),2.1 (233),3.1 (226),2.9 (100),0.5 (219),0.8 (76),1.7 (142),7.1 (236),9.1 (228)
239,Jordan Tucker,DEL,18.3,2.7 (228),7.6 (187),0.358 (250),1.3 (142),5.1 (109),0.25 (222),0.1 (263),0.3 (254),0.5 (250),0.7 (205),2.0 (240),2.7 (240),0.7 (252),0.1 (266),0.4 (149),0.3 (1),7.0 (239),5.7 (262)
242,Patrick McCaw,DEL,24.6,2.5 (241),5.9 (235),0.429 (168),0.9 (189),3.2 (175),0.286 (190),0.5 (205),0.5 (238),1.0 (1),1.0 (150),2.1 (233),3.1 (226),2.3 (133),1.7 (28),0.0 (263),1.2 (60),6.8 (242),9.3 (226)


In [7]:
import requests
import pandas as pd

# API URL for Advanced Player Stats
url = "https://stats.gleague.nba.com/stats/leaguedashplayerstatscombined"
params = {
    "LeagueID": "20",
    "PerMode": "PerGame",
    "MeasureType": "Advanced",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "PaceAdjust": "N",
    "Rank": "N",
    "Outcome": None,
    "Location": None,
    "Month": 0,
    "SeasonSegment": None,
    "DateFrom": None,
    "DateTo": None,
    "OpponentTeamID": 0,
    "VsConference": None,
    "VsDivision": None,
    "TeamID": 0,
    "Division": None,
    "PlayerPosition": None,
    "StarterBench": None,
    "TwoWay": 0,
}

# Headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Connection": "keep-alive"
}

# Initialize stats as None
stats = None

# Fetching the data
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()

    # Extracting headers and rows
    headers = data['resultSets'][0]['headers']
    rows = data['resultSets'][0]['rowSet']

    # Creating a DataFrame
    stats = pd.DataFrame(rows, columns=headers)

# Display stats
if stats is not None:
    print(stats)
else:
    print("No data available or API request failed.")

No data available or API request failed.


In [ ]:
# Drop unnecessary columns
    columns_to_drop = ["PLAYER_ID", "RANK", "TEAM_ID", "GP"]
    stats = stats.drop(columns=[col for col in columns_to_drop if col in stats.columns])

    # Rename columns
    stats = stats.rename(columns={
        "FG_PCT": "FG%",
        "FT_PCT": "FT%",
        "FG3M": "3PM",
        "FG3A": "3PA",
        "FG3_PCT": "3P%"
    })

    # Round the MIN column to one decimal point
    if "MIN" in stats.columns:
        stats["MIN"] = stats["MIN"].round(1)

    # Filter for players who play at least 15 minutes per game
    filtered_stats = stats[stats["MIN"] >= 15]

    # Identify numeric columns for formatting, excluding "MIN"
    numeric_cols = [col for col in stats.columns if col not in ["MIN"] and stats[col].dtype in ['float64', 'int64']]

    # Add ranks to numeric columns based on all players who play at least 15 minutes
    df_with_ranks = stats.copy()
    ranks_data = {}
    for col in numeric_cols:
        if col == "TOV":
            # Inverted ranking for PF (lower is better)
            ranks = filtered_stats[col].rank(ascending=True, method='min').astype(int)
        else:
            # Higher values are better for other columns
            ranks = filtered_stats[col].rank(ascending=False, method='min').astype(int)
        # Remove parentheses for players not meeting the threshold
        df_with_ranks[col] = stats[col].astype(str) + df_with_ranks.index.map(
            lambda idx: f" ({ranks[idx]})" if idx in ranks.index else ""
        )
        ranks_data[col] = ranks

    # Compute color scaling based on ranks
    rank_min = 1
    rank_max = filtered_stats.shape[0]  # Maximum rank is total filtered players

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            rank_str = row[col]
            if " (" not in rank_str:  # No rank for players below the threshold
                colors.append("background-color: none; color: black;")
                continue
            rank = int(rank_str.split("(")[-1].replace(")", ""))  # Extract the rank from the string
            # Normalize rank for color mapping
            norm = (rank - rank_min) / (rank_max - rank_min) if rank_max > rank_min else 0.5
            # Map normalized rank to color
            color = mcolors.to_hex(plt.cm.RdYlGn(1 - norm))  # Invert scale for ranks (lower is better)
            # Adjust text color for readability
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Apply RAP team filter for display
    stats = stats[stats["TEAM"] == "DEL"]
    df_with_ranks = df_with_ranks[df_with_ranks["TEAM"] == "DEL"]

    # Apply styling
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Ensure MIN is displayed with one decimal precision
    if "MIN" in df_with_ranks.columns:
        styled_df = styled_df.format({"MIN": "{:.1f}"})

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

In [11]:
import requests
import pandas as pd

# API URL
url = "https://stats.gleague.nba.com/stats/leagueLeaders"
params = {
    "LeagueID": "20",
    "PerMode": "PerGame",
    "StatCategory": "PTS",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "Scope": "S"
}

# Headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Connection": "keep-alive"
}

# Fetching the data
response = requests.get(url, headers=headers, params=params)
data = response.json()

# Extracting headers and rows
headers = data['resultSet']['headers']
rows = data['resultSet']['rowSet']

# Creating a DataFrame
df = pd.DataFrame(rows, columns=headers)

# Saving data to a CSV file
df.to_csv('GLeague_Stats.csv', index=False)

# Displaying first 5 rows
print(df.head())

   PLAYER_ID  RANK            PLAYER     TEAM_ID TEAM  GP   MIN   FGM   FGA  \
0    1631121     1    Bryce McGowens  1612709933  RCR  11  35.3   9.6  19.6   
1    1630314     2  Brandon Williams  1612709918  TEX   8  37.9   9.9  20.8   
2    1631120     3        JD Davison  1612709915  MNE  12  35.0   9.4  20.2   
3     203933     4       T.J. Warren  1612709919  WES  10  36.6  10.6  20.9   
4    1630528     5  Josh Christopher  1612709904  SXF  12  35.7   9.3  20.0   

   FG_PCT  ...  FT_PCT  OREB  DREB  REB  AST  STL  BLK  TOV   PTS   EFF  
0   0.491  ...   0.893   0.9   2.5  3.5  3.3  1.5  0.7  3.6  29.4  24.1  
1   0.476  ...   0.667   1.8   3.9  5.6  6.8  2.0  0.8  4.6  28.5  26.5  
2   0.467  ...   0.780   1.3   3.7  5.0  8.8  1.5  1.3  2.9  27.3  29.0  
3   0.507  ...   0.750   3.9   4.2  8.1  3.5  1.9  0.7  2.4  27.2  28.1  
4   0.467  ...   0.840   0.9   5.2  6.1  4.5  2.4  0.7  3.1  24.6  24.2  

[5 rows x 25 columns]


In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashteamshotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "Conference": "",
    "DateFrom": "",
    "DateTo": "",
    "DistanceRange": "By Zone",
    "Division": "",
    "GameScope": "",
    "GameSegment": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Opponent",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Extract the relevant data
    if "resultSets" in data and "rowSet" in data["resultSets"]:
        result_set = data["resultSets"]["rowSet"]
        headers = data["resultSets"]["headers"]

        # Extract column names
        column_names = headers[1]["columnNames"]

        # Convert to a DataFrame
        team_stats = pd.DataFrame(result_set, columns=column_names)

        # Drop the TEAM_ID column
        if "TEAM_ID" in team_stats.columns:
            team_stats = team_stats.drop(columns=["TEAM_ID"])

        # Rename OPP_FGM, OPP_FGA, and OPP_FG% to FGM, FGA, and FG%
        team_stats = team_stats.rename(columns=lambda x: x.replace("OPP_FGM", "FGM")
                                                         .replace("OPP_FGA", "FGA")
                                                         .replace("OPP_FG_PCT", "FG%"))
        team_stats = team_stats.rename(columns={"TEAM_NAME": "TEAM"})

        # Add top-level column names to create a MultiIndex
        top_level = [" "] + ["Restricted Area", "Restricted Area", "Restricted Area",
                             "In The Paint (Non-RA)", "In The Paint (Non-RA)", "In The Paint (Non-RA)",
                             "Mid-Range", "Mid-Range", "Mid-Range",
                             "Left Corner 3", "Left Corner 3", "Left Corner 3",
                             "Right Corner 3", "Right Corner 3", "Right Corner 3",
                             "Above the Break 3", "Above the Break 3", "Above the Break 3",
                             "Backcourt", "Backcourt", "Backcourt",
                             "Corner 3", "Corner 3", "Corner 3"]

        if len(top_level) != len(team_stats.columns):
            raise ValueError("Mismatch between top-level column structure and actual data columns.")

        # Combine the new top-level columns with the current column names
        multi_index = pd.MultiIndex.from_tuples(zip(top_level, team_stats.columns))
        team_stats.columns = multi_index

        # Drop the "Backcourt" columns
        team_stats = team_stats.drop(columns=["Backcourt"], level=0)

        # Rearrange columns: move "Left Corner 3" and "Right Corner 3" to the right of "Corner 3"
        column_order = (
            [col for col in team_stats.columns if col[0] not in ["Left Corner 3", "Right Corner 3"]] +
            [col for col in team_stats.columns if col[0] in ["Left Corner 3", "Right Corner 3"]]
        )
        team_stats = team_stats[column_order]

        # Apply conditional formatting and ranking
        numeric_cols = [col for col in team_stats.columns if col[1] in ["FGM", "FGA", "FG%"]]
        df_with_ranks = team_stats.copy()

        for col in numeric_cols:
            # Set ascending=True for ascending order ranks
            ranks = team_stats[col].rank(ascending=True, method='min').astype(int)
            df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

        col_min = team_stats[numeric_cols].min()
        col_max = team_stats[numeric_cols].max()

        def luminance(color):
            rgb = mcolors.hex2color(color)
            return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

        def color_cells(row):
            colors = []
            for col in numeric_cols:
                value = team_stats.loc[row.name, col]
                # Reverse normalization for ascending order
                norm = (col_max[col] - value) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
                color = mcolors.to_hex(plt.cm.RdYlGn(norm))
                text_color = 'white' if luminance(color) < 0.5 else 'black'
                colors.append(f"background-color: {color}; color: {text_color};")
            return colors

        # Apply styling
        styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

        # Display the styled DataFrame
        from IPython.display import display
        display(styled_df)

    else:
        print("No valid data found in the response.")
else:
    print(f"Request failed with status code {response.status_code}")

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import time

# Set up Selenium WebDriver
service = Service(r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe")  # Update with your path to chromedriver

#Mac file path
#"/Users/fadihaddad/Documents/chromedriver-mac-arm64/chromedriver"

#PC file path
#r"C:\Users\PC\OneDrive\Documents\GitHub\chromedriver-win64\chromedriver.exe"

options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://stats.gleague.nba.com/leaders/"

# Load the webpage
driver.get(url)

# Allow time for JavaScript to load
time.sleep(10)  # Adjust the time as per your internet speed and website response time

# Locate the table
table_xpath = '//div[contains(@class, "nba-stat-table__overflow")]/table'
table_element = driver.find_element(By.XPATH, table_xpath)

# Extract the table data
table_html = table_element.get_attribute('outerHTML')

# Convert the table HTML into a pandas DataFrame
player_stats = pd.read_html(table_html)[0]

# Quit the driver
driver.quit()

player_stats

,#,Player,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF
0,1,Bryce McGowens,11,35.3,29.4,9.6,19.6,49.1,2.5,7.2,...,5.1,89.3,0.9,2.5,3.5,3.3,1.5,0.7,3.6,24.1
1,2,Brandon Williams,8,37.9,28.5,9.9,20.8,47.6,3.3,8.4,...,4.9,66.7,1.8,3.9,5.6,6.8,2.0,0.8,4.6,26.5
2,3,JD Davison,11,35.1,27.5,9.5,19.9,47.5,1.7,5.8,...,5.2,77.2,1.2,3.1,4.3,8.9,1.6,1.1,3.1,28.6
3,4,T.J. Warren,10,36.6,27.2,10.6,20.9,50.7,2.8,7.7,...,2.4,75.0,3.9,4.2,8.1,3.5,1.9,0.7,2.4,28.1
4,5,Cole Swider,7,33.9,24.7,8.4,16.4,51.3,5.4,10.9,...,1.1,100.0,1.3,3.7,5.0,2.6,1.6,0.6,1.6,24.9
5,6,Josh Christopher,12,35.7,24.6,9.3,20.0,46.7,3.1,9.8,...,2.1,84.0,0.9,5.2,6.1,4.5,2.4,0.7,3.1,24.2
6,7,Keshad Johnson,9,35.1,24.3,8.7,15.1,57.4,1.6,3.9,...,3.7,81.8,3.0,5.8,8.8,1.4,1.2,0.8,1.3,28.1
7,8,Isaiah Wong,8,31.4,24.1,7.9,17.0,46.3,2.0,4.9,...,4.9,84.6,1.3,2.6,3.9,3.6,1.5,0.4,2.5,21.1
8,9,A.J. Lawson,11,35.1,24.0,7.7,17.6,43.8,2.5,7.9,...,3.7,82.9,0.9,5.7,6.6,2.5,1.2,0.3,1.6,22.5
9,10,Jeff Dowtin Jr.,8,33.3,23.6,9.3,18.3,50.7,2.5,6.0,...,2.3,83.3,0.3,2.8,3.0,4.6,1.1,0.5,1.6,21.9


In [18]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the URL for the data request
url = "https://stats.gleague.nba.com/stats/leaguedashteamshotlocations"

# Define the headers for the network request
headers = {
    "Host": "stats.gleague.nba.com",
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "x-nba-stats-origin": "stats",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Origin": "https://stats.gleague.nba.com",
    "Referer": "https://stats.gleague.nba.com/",
}

# Define the query parameters
params = {
    "Conference": "",
    "DateFrom": "",
    "DateTo": "",
    "DistanceRange": "8ft Range",
    "Division": "",
    "GameScope": "",
    "GameSegment": "",
    "LastNGames": "0",
    "LeagueID": "20",
    "Location": "",
    "MeasureType": "Opponent",
    "Month": "0",
    "OpponentTeamID": "0",
    "Outcome": "",
    "PORound": "0",
    "PaceAdjust": "N",
    "PerMode": "PerGame",
    "Period": "0",
    "PlayerExperience": "",
    "PlayerPosition": "",
    "PlusMinus": "N",
    "Rank": "N",
    "Season": "2024-25",
    "SeasonType": "Showcase",
    "ShotClockRange": "",
    "StarterBench": "",
    "TeamID": "0",
    "VsConference": "",
    "VsDivision": "",
}

# Send the GET request
response = requests.get(url, headers=headers, params=params)

# Verify if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse the JSON data

    # Debug: Inspect JSON structure
    print("Debug: JSON Response Structure:\n", data)

    # Extract the relevant data
    result_set = data["resultSets"]["rowSet"]
    columns = data["resultSets"]["headers"][1]["columnNames"]

    # Convert to a DataFrame
    team_stats = pd.DataFrame(result_set, columns=columns)

    # Drop the TEAM_ID column
    if "TEAM_ID" in team_stats.columns:
        team_stats = team_stats.drop(columns=["TEAM_ID"])

    # Rename FG_PCT columns to FG% and TEAM_NAME column to TEAM
    team_stats = team_stats.rename(columns=lambda x: x.replace("FG_PCT", "FG%") if "FG_PCT" in x else x)
    team_stats = team_stats.rename(columns={"TEAM_NAME": "TEAM"})

    # Add top-level column names to create a MultiIndex
    top_level = [" "] + ["Less Than 8ft.", "Less Than 8ft.", "Less Than 8ft.",
                         "8-16 ft.", "8-16 ft.", "8-16 ft.",
                         "16-24 ft.", "16-24 ft.", "16-24 ft.",
                         "24+ ft.", "24+ ft.", "24+ ft.",
                         "Back Court Shot", "Back Court Shot", "Back Court Shot"]

    # Ensure alignment between columns and top-level
    if len(top_level) != len(team_stats.columns):
        raise ValueError("Mismatch between top-level column structure and actual data columns.")

    # Combine the new top level with the current column names
    multi_index = pd.MultiIndex.from_tuples(zip(top_level, team_stats.columns))

    # Apply the MultiIndex to the DataFrame
    team_stats.columns = multi_index

    # Apply conditional formatting and ranking
    numeric_cols = [col for col in team_stats.columns if col[1] in ["FGM", "FGA", "FG%"]]
    df_with_ranks = team_stats.copy()

    # Add ranks
    for col in numeric_cols:
        ranks = team_stats[col].rank(ascending=False, method='min').astype(int)
        df_with_ranks[col] = team_stats[col].astype(str) + ' (' + ranks.astype(str) + ')'

    # Apply color formatting
    col_min = team_stats[numeric_cols].min()
    col_max = team_stats[numeric_cols].max()

    def luminance(color):
        rgb = mcolors.hex2color(color)
        return 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]

    def color_cells(row):
        colors = []
        for col in numeric_cols:
            value = team_stats.loc[row.name, col]  # Use original numeric value for coloring
            norm = (value - col_min[col]) / (col_max[col] - col_min[col]) if col_min[col] != col_max[col] else 0.5
            color = mcolors.to_hex(plt.cm.RdYlGn(norm))
            text_color = 'white' if luminance(color) < 0.5 else 'black'
            colors.append(f"background-color: {color}; color: {text_color};")
        return colors

    # Style the DataFrame
    styled_df = df_with_ranks.style.apply(color_cells, subset=numeric_cols, axis=1).format(precision=3)

    # Display the styled DataFrame
    from IPython.display import display
    display(styled_df)

else:
    print(f"Request failed with status code {response.status_code}")

Debug: JSON Response Structure:
 {'resource': 'leaguedashteamshotlocations', 'parameters': {'MeasureType': 'Opponent', 'PerMode': 'PerGame', 'PlusMinus': 'N', 'PaceAdjust': 'N', 'Rank': 'N', 'LeagueID': '20', 'Season': '2024-25', 'SeasonType': 'Showcase', 'PORound': 0, 'Outcome': None, 'Location': None, 'Month': 0, 'SeasonSegment': None, 'DateFrom': None, 'DateTo': None, 'OpponentTeamID': 0, 'VsConference': None, 'VsDivision': None, 'TeamID': 0, 'Conference': None, 'Division': None, 'GameSegment': None, 'Period': 0, 'ShotClockRange': None, 'LastNGames': 0, 'DistanceRange': '8ft Range', 'GameScope': None, 'PlayerExperience': None, 'PlayerPosition': None, 'StarterBench': None, 'ISTRound': None}, 'resultSets': {'name': 'ShotLocations', 'headers': [{'name': 'SHOT_CATEGORY', 'columnsToSkip': 2, 'columnSpan': 3, 'columnNames': ['Less Than 8 ft.', '8-16 ft.', '16-24 ft.', '24+ ft.', 'Back Court Shot']}, {'name': 'columns', 'columnSpan': 1, 'columnNames': ['TEAM_ID', 'TEAM_NAME', 'OPP_FGM', 'O

In [7]:
print(team_stats.columns)

MultiIndex([(             ' ', 'Team'),
            ('Less than 5ft.',  'FGM'),
            ('Less than 5ft.',  'FGA'),
            ('Less than 5ft.',  'FG%'),
            (       '5-9 ft.',  'FGM'),
            (       '5-9 ft.',  'FGA'),
            (       '5-9 ft.',  'FG%'),
            (     '10-14 ft.',  'FGM'),
            (     '10-14 ft.',  'FGA'),
            (     '10-14 ft.',  'FG%'),
            (     '15-19 ft.',  'FGM'),
            (     '15-19 ft.',  'FGA'),
            (     '15-19 ft.',  'FG%'),
            (     '20-24 ft.',  'FGM'),
            (     '20-24 ft.',  'FGA'),
            (     '20-24 ft.',  'FG%'),
            (     '25-29 ft.',  'FGM'),
            (     '25-29 ft.',  'FGA'),
            (     '25-29 ft.',  'FG%')],
           )


In [35]:
# Print the columns of the DataFrame and their respective MultiIndex levels
if isinstance(team_stats.columns, pd.MultiIndex):
    # If the columns are a MultiIndex, display each level and its corresponding labels
    print("MultiIndex Columns:")
    for idx, col in enumerate(team_stats.columns.tolist()):
        print(f"Column {idx + 1}: {col} (Level 0: '{col[0]}', Level 1: '{col[1]}')")
else:
    # If not a MultiIndex, just display the column names
    print("Single Index Columns:")
    print(team_stats.columns.tolist())

MultiIndex Columns:
Column 1: ('Unnamed: 0_level_0', 'Unnamed: 0_level_1', 'Unnamed: 0_level_2', 'TEAM', 'TEAM', 'TEAM') (Level 0: 'Unnamed: 0_level_0', Level 1: 'Unnamed: 0_level_1')
Column 2: ('Less\xa0than\xa05ft.', 'Less\xa0Than\xa08ft.', 'Restricted\xa0Area', 'FGM', 'FGM', 'FGM') (Level 0: 'Less than 5ft.', Level 1: 'Less Than 8ft.')
Column 3: ('Less\xa0than\xa05ft.', 'Less\xa0Than\xa08ft.', 'Restricted\xa0Area', 'FGA', 'FGA', 'FGA') (Level 0: 'Less than 5ft.', Level 1: 'Less Than 8ft.')
Column 4: ('Less\xa0than\xa05ft.', 'Less\xa0Than\xa08ft.', 'Restricted\xa0Area', 'FG%', 'FG%', 'FG%') (Level 0: 'Less than 5ft.', Level 1: 'Less Than 8ft.')
Column 5: ('5-9\xa0ft.', '8-16\xa0ft.', 'In\xa0The\xa0Paint (Non-RA)', 'FGM', 'FGM', 'FGM') (Level 0: '5-9 ft.', Level 1: '8-16 ft.')
Column 6: ('5-9\xa0ft.', '8-16\xa0ft.', 'In\xa0The\xa0Paint (Non-RA)', 'FGA', 'FGA', 'FGA') (Level 0: '5-9 ft.', Level 1: '8-16 ft.')
Column 7: ('5-9\xa0ft.', '8-16\xa0ft.', 'In\xa0The\xa0Paint (Non-RA)', 'FG%',

In [37]:
# Drop all columns that contain "Unnamed" in any level of their MultiIndex
columns_to_drop = [col for col in team_stats.columns if any("Unnamed" in str(level) for level in col)]
team_stats = team_stats.drop(columns=columns_to_drop)

# Print the remaining columns to verify
print("Remaining Columns:", team_stats.columns.tolist())

Remaining Columns: [('Less\xa0than\xa05ft.', 'Less\xa0Than\xa08ft.', 'Restricted\xa0Area', 'FGM', 'FGM', 'FGM'), ('Less\xa0than\xa05ft.', 'Less\xa0Than\xa08ft.', 'Restricted\xa0Area', 'FGA', 'FGA', 'FGA'), ('Less\xa0than\xa05ft.', 'Less\xa0Than\xa08ft.', 'Restricted\xa0Area', 'FG%', 'FG%', 'FG%'), ('5-9\xa0ft.', '8-16\xa0ft.', 'In\xa0The\xa0Paint (Non-RA)', 'FGM', 'FGM', 'FGM'), ('5-9\xa0ft.', '8-16\xa0ft.', 'In\xa0The\xa0Paint (Non-RA)', 'FGA', 'FGA', 'FGA'), ('5-9\xa0ft.', '8-16\xa0ft.', 'In\xa0The\xa0Paint (Non-RA)', 'FG%', 'FG%', 'FG%'), ('10-14\xa0ft.', '16-24\xa0ft.', 'Mid-Range', 'FGM', 'FGM', 'FGM'), ('10-14\xa0ft.', '16-24\xa0ft.', 'Mid-Range', 'FGA', 'FGA', 'FGA'), ('10-14\xa0ft.', '16-24\xa0ft.', 'Mid-Range', 'FG%', 'FG%', 'FG%'), ('15-19\xa0ft.', '24+\xa0ft.', 'Left\xa0Corner\xa03', 'FGM', 'FGM', 'FGM'), ('15-19\xa0ft.', '24+\xa0ft.', 'Left\xa0Corner\xa03', 'FGA', 'FGA', 'FGA'), ('15-19\xa0ft.', '24+\xa0ft.', 'Left\xa0Corner\xa03', 'FG%', 'FG%', 'FG%'), ('20-24\xa0ft.', 'Ba

In [44]:
team_stats.columns

Index(['Unnamed: 0_level_0 Unnamed: 0_level_1',
       'Less than 5ft. Less Than 8ft.', 'Less than 5ft. Less Than 8ft..1',
       'Less than 5ft. Less Than 8ft..2', '5-9 ft. 8-16 ft.',
       '5-9 ft. 8-16 ft..1', '5-9 ft. 8-16 ft..2', '10-14 ft. 16-24 ft.',
       '10-14 ft. 16-24 ft..1', '10-14 ft. 16-24 ft..2', '15-19 ft. 24+ ft.',
       '15-19 ft. 24+ ft..1', '15-19 ft. 24+ ft..2',
       '20-24 ft. Back Court Shot', '20-24 ft. Back Court Shot.1',
       '20-24 ft. Back Court Shot.2', '25-29 ft. Unnamed: 16_level_1',
       '25-29 ft. Unnamed: 17_level_1', '25-29 ft. Unnamed: 18_level_1'],
      dtype='object')